In [10]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [11]:
#apsrtc
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
route_links=[]
data = []

# Initialize WebDriver
driver = webdriver.Chrome()

try:
    driver.get("https://www.redbus.in/online-booking/apsrtc/?utm_source=rtchometile")
    driver.maximize_window()
    
    # Initialize WebDriverWait
    wait = WebDriverWait(driver, 10)  # 10 seconds timeout
    
    # Wait for elements with the class name "route" to be present
    route_elements = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "route")))
    
    for route in route_elements:
        try:
            route_href = route.get_attribute('href')
            route_title = route.get_attribute('title')
            route_links.append(route_href)
            data.append({"route_link": route_href, "route_title": route_title})
        except TimeoutException as e:
            print("TimeoutException caught:", e)


finally:
    driver.quit()
# Print the collected data
for item in data:
    print(item)



{'route_link': 'https://www.redbus.in/bus-tickets/vijayawada-to-hyderabad', 'route_title': 'Vijayawada to Hyderabad'}
{'route_link': 'https://www.redbus.in/bus-tickets/hyderabad-to-vijayawada', 'route_title': 'Hyderabad to Vijayawada'}
{'route_link': 'https://www.redbus.in/bus-tickets/kakinada-to-visakhapatnam', 'route_title': 'Kakinada to Visakhapatnam'}
{'route_link': 'https://www.redbus.in/bus-tickets/visakhapatnam-to-kakinada', 'route_title': 'Visakhapatnam to Kakinada'}
{'route_link': 'https://www.redbus.in/bus-tickets/chittoor-andhra-pradesh-to-bangalore', 'route_title': 'Chittoor (Andhra Pradesh) to Bangalore'}
{'route_link': 'https://www.redbus.in/bus-tickets/kadapa-to-bangalore', 'route_title': 'Kadapa to Bangalore'}
{'route_link': 'https://www.redbus.in/bus-tickets/ananthapur-to-bangalore', 'route_title': 'Anantapur (andhra pradesh) to Bangalore'}
{'route_link': 'https://www.redbus.in/bus-tickets/tirupathi-to-bangalore', 'route_title': 'Tirupati to Bangalore'}
{'route_link': 

In [12]:
route_links

['https://www.redbus.in/bus-tickets/vijayawada-to-hyderabad',
 'https://www.redbus.in/bus-tickets/hyderabad-to-vijayawada',
 'https://www.redbus.in/bus-tickets/kakinada-to-visakhapatnam',
 'https://www.redbus.in/bus-tickets/visakhapatnam-to-kakinada',
 'https://www.redbus.in/bus-tickets/chittoor-andhra-pradesh-to-bangalore',
 'https://www.redbus.in/bus-tickets/kadapa-to-bangalore',
 'https://www.redbus.in/bus-tickets/ananthapur-to-bangalore',
 'https://www.redbus.in/bus-tickets/tirupathi-to-bangalore',
 'https://www.redbus.in/bus-tickets/visakhapatnam-to-vijayawada',
 'https://www.redbus.in/bus-tickets/ongole-to-hyderabad']

In [13]:
df=pd.DataFrame(data)
df.reset_index(drop=True)

,route_link,route_title
0,https://www.redbus.in/bus-tickets/vijayawada-t...,Vijayawada to Hyderabad
1,https://www.redbus.in/bus-tickets/hyderabad-to...,Hyderabad to Vijayawada
2,https://www.redbus.in/bus-tickets/kakinada-to-...,Kakinada to Visakhapatnam
3,https://www.redbus.in/bus-tickets/visakhapatna...,Visakhapatnam to Kakinada
4,https://www.redbus.in/bus-tickets/chittoor-and...,Chittoor (Andhra Pradesh) to Bangalore
5,https://www.redbus.in/bus-tickets/kadapa-to-ba...,Kadapa to Bangalore
6,https://www.redbus.in/bus-tickets/ananthapur-t...,Anantapur (andhra pradesh) to Bangalore
7,https://www.redbus.in/bus-tickets/tirupathi-to...,Tirupati to Bangalore
8,https://www.redbus.in/bus-tickets/visakhapatna...,Visakhapatnam to Vijayawada
9,https://www.redbus.in/bus-tickets/ongole-to-hy...,Ongole to Hyderabad


In [14]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import traceback

def scrape_bus_details(driver, route_links):
    details_list = []

    for link in route_links:
        driver.get(link)
        time.sleep(2)  # Allow the page to load initially
        
        # Get the body element for scrolling
        body = driver.find_element(By.TAG_NAME, "body")

        # Optional: Click on "View Buses" if needed
        try:
            view_buses = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, "//div[@class='button']"))
            )
            driver.execute_script("arguments[0].click();", view_buses)
            print("Clicked View Buses")
        except Exception as e:
            print("No 'View Buses' button found, continuing to scrape details...")
            traceback.print_exc()

        # Scrolling logic
        scrolling = True
        while scrolling:
            old_page_source = driver.page_source

            # Scroll down
            body.send_keys(Keys.PAGE_DOWN)
            time.sleep(2)  # Wait for new data to load

            new_page_source = driver.page_source

            if new_page_source == old_page_source:
                scrolling = False

        # Collect bus details
        wait = WebDriverWait(driver, 30)
        bus_containers = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "bus-item-details")))

        for container in bus_containers:
            try:
                bus_name = container.find_element(By.XPATH, ".//div[@class='travels lh-24 f-bold d-color']").text
                bus_type = container.find_element(By.XPATH, ".//div[@class='bus-type f-12 m-top-16 l-color evBus']").text
                departing_time = container.find_element(By.XPATH, ".//div[@class='dp-time f-19 d-color f-bold']").text
                duration = container.find_element(By.XPATH, ".//div[@class='dur l-color lh-24']").text
                reaching_time = container.find_element(By.XPATH, ".//div[@class='bp-time f-19 d-color disp-Inline']").text

                try:
                    rating = container.find_element(By.CSS_SELECTOR, ".rating-sec .rating span").text
                except Exception as e:
                    rating = "N/A"  # Set a default value if rating is not found
                    print(f"Error extracting rating: {e}")
                    traceback.print_exc()

                price = container.find_element(By.XPATH, ".//div[contains(@class, 'fare d-block')]").text
                seats_available = container.find_element(By.CLASS_NAME, "seat-left").text

                details_list.append({
                    "bus_name": bus_name,
                    "bus_type": bus_type,
                    "departing_time": departing_time,
                    "duration": duration,
                    "reaching_time": reaching_time,
                    "rating": rating,
                    "price": price,
                    "seats_available": seats_available
                })

            except Exception as e:
                print(f"Error extracting details from container: {e}")
                traceback.print_exc()

    return details_list

# Initialize the WebDriver
driver = webdriver.Chrome()

# Example list of route links
route_links = [
    'https://www.redbus.in/bus-tickets/vijayawada-to-hyderabad',
    'https://www.redbus.in/bus-tickets/hyderabad-to-vijayawada',
    'https://www.redbus.in/bus-tickets/kakinada-to-visakhapatnam',
    'https://www.redbus.in/bus-tickets/visakhapatnam-to-kakinada',
    'https://www.redbus.in/bus-tickets/chittoor-andhra-pradesh-to-bangalore',
    'https://www.redbus.in/bus-tickets/kadapa-to-bangalore',
    'https://www.redbus.in/bus-tickets/ananthapur-to-bangalore',
    'https://www.redbus.in/bus-tickets/tirupathi-to-bangalore',
    'https://www.redbus.in/bus-tickets/visakhapatnam-to-vijayawada',
    'https://www.redbus.in/bus-tickets/ongole-to-hyderabad'
]

# Perform scraping
details_list = scrape_bus_details(driver, route_links)

# Print the collected details
print(details_list)

# Close the browser
driver.quit()


Clicked View Buses
Clicked View Buses
Clicked View Buses
Clicked View Buses
Clicked View Buses
Clicked View Buses
Clicked View Buses
Clicked View Buses
Clicked View Buses
Clicked View Buses
[{'bus_name': 'APSRTC - 35133', 'bus_type': 'AMARAVATHI (VOLVO / SCANIA A.C Multi Axle)', 'departing_time': '17:00', 'duration': '05h 50m', 'reaching_time': '22:50', 'rating': '3.3', 'price': 'INR 720', 'seats_available': '33 Seats available'}, {'bus_name': 'APSRTC - 35085', 'bus_type': 'DOLPHIN CRUISE (VOLVO / SCANIA A.C Multi Axle)', 'departing_time': '18:30', 'duration': '06h 00m', 'reaching_time': '00:30', 'rating': '3.9', 'price': 'INR 720', 'seats_available': '38 Seats available'}, {'bus_name': 'APSRTC - 3613', 'bus_type': 'Super Luxury (Non AC Seater 2+2 Push Back)', 'departing_time': '19:00', 'duration': '07h 42m', 'reaching_time': '02:42', 'rating': '3.1', 'price': 'INR 469', 'seats_available': '33 Seats available'}, {'bus_name': 'APSRTC - 9316', 'bus_type': 'Super Luxury (Non AC Seater 2+2

In [15]:
import pandas as pd
df1=pd.DataFrame(details_list)

df1

,bus_name,bus_type,departing_time,duration,reaching_time,rating,price,seats_available
0,APSRTC - 35133,AMARAVATHI (VOLVO / SCANIA A.C Multi Axle),17:00,05h 50m,22:50,3.3,INR 720,33 Seats available
1,APSRTC - 35085,DOLPHIN CRUISE (VOLVO / SCANIA A.C Multi Axle),18:30,06h 00m,00:30,3.9,INR 720,38 Seats available
2,APSRTC - 3613,Super Luxury (Non AC Seater 2+2 Push Back),19:00,07h 42m,02:42,3.1,INR 469,33 Seats available
3,APSRTC - 9316,Super Luxury (Non AC Seater 2+2 Push Back),20:20,07h 50m,04:10,4.1,INR 469,16 Seats available
4,APSRTC - 35189,DOLPHIN CRUISE (VOLVO / SCANIA A.C Multi Axle),20:30,04h 35m,01:05,3.8,INR 670,27 Seats available
...,...,...,...,...,...,...,...,...
140,,,,,,,,
141,,,,,,,,
142,,,,,,,,
143,,,,,,,,


In [16]:
result = pd.concat([df, df1], axis=1)
result

,route_link,route_title,bus_name,bus_type,departing_time,duration,reaching_time,rating,price,seats_available
0,https://www.redbus.in/bus-tickets/vijayawada-t...,Vijayawada to Hyderabad,APSRTC - 35133,AMARAVATHI (VOLVO / SCANIA A.C Multi Axle),17:00,05h 50m,22:50,3.3,INR 720,33 Seats available
1,https://www.redbus.in/bus-tickets/hyderabad-to...,Hyderabad to Vijayawada,APSRTC - 35085,DOLPHIN CRUISE (VOLVO / SCANIA A.C Multi Axle),18:30,06h 00m,00:30,3.9,INR 720,38 Seats available
2,https://www.redbus.in/bus-tickets/kakinada-to-...,Kakinada to Visakhapatnam,APSRTC - 3613,Super Luxury (Non AC Seater 2+2 Push Back),19:00,07h 42m,02:42,3.1,INR 469,33 Seats available
3,https://www.redbus.in/bus-tickets/visakhapatna...,Visakhapatnam to Kakinada,APSRTC - 9316,Super Luxury (Non AC Seater 2+2 Push Back),20:20,07h 50m,04:10,4.1,INR 469,16 Seats available
4,https://www.redbus.in/bus-tickets/chittoor-and...,Chittoor (Andhra Pradesh) to Bangalore,APSRTC - 35189,DOLPHIN CRUISE (VOLVO / SCANIA A.C Multi Axle),20:30,04h 35m,01:05,3.8,INR 670,27 Seats available
...,...,...,...,...,...,...,...,...,...,...
140,NaN,NaN,,,,,,,,
141,NaN,NaN,,,,,,,,
142,NaN,NaN,,,,,,,,
143,NaN,NaN,,,,,,,,


In [17]:
result = result.ffill()  # Forward fill
result

,route_link,route_title,bus_name,bus_type,departing_time,duration,reaching_time,rating,price,seats_available
0,https://www.redbus.in/bus-tickets/vijayawada-t...,Vijayawada to Hyderabad,APSRTC - 35133,AMARAVATHI (VOLVO / SCANIA A.C Multi Axle),17:00,05h 50m,22:50,3.3,INR 720,33 Seats available
1,https://www.redbus.in/bus-tickets/hyderabad-to...,Hyderabad to Vijayawada,APSRTC - 35085,DOLPHIN CRUISE (VOLVO / SCANIA A.C Multi Axle),18:30,06h 00m,00:30,3.9,INR 720,38 Seats available
2,https://www.redbus.in/bus-tickets/kakinada-to-...,Kakinada to Visakhapatnam,APSRTC - 3613,Super Luxury (Non AC Seater 2+2 Push Back),19:00,07h 42m,02:42,3.1,INR 469,33 Seats available
3,https://www.redbus.in/bus-tickets/visakhapatna...,Visakhapatnam to Kakinada,APSRTC - 9316,Super Luxury (Non AC Seater 2+2 Push Back),20:20,07h 50m,04:10,4.1,INR 469,16 Seats available
4,https://www.redbus.in/bus-tickets/chittoor-and...,Chittoor (Andhra Pradesh) to Bangalore,APSRTC - 35189,DOLPHIN CRUISE (VOLVO / SCANIA A.C Multi Axle),20:30,04h 35m,01:05,3.8,INR 670,27 Seats available
...,...,...,...,...,...,...,...,...,...,...
140,https://www.redbus.in/bus-tickets/ongole-to-hy...,Ongole to Hyderabad,,,,,,,,
141,https://www.redbus.in/bus-tickets/ongole-to-hy...,Ongole to Hyderabad,,,,,,,,
142,https://www.redbus.in/bus-tickets/ongole-to-hy...,Ongole to Hyderabad,,,,,,,,
143,https://www.redbus.in/bus-tickets/ongole-to-hy...,Ongole to Hyderabad,,,,,,,,


**TSRTC**

In [18]:
#apsrtc
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
route_links=[]
data = []

# Initialize WebDriver
driver = webdriver.Chrome()

try:
    driver.get("https://www.redbus.in/online-booking/tsrtc/?utm_source=rtchometile")
    driver.maximize_window()
    
    # Initialize WebDriverWait
    wait = WebDriverWait(driver, 10)  # 10 seconds timeout
    
    # Wait for elements with the class name "route" to be present
    route_elements = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "route")))
    
    for route in route_elements:
        try:
            route_href = route.get_attribute('href')
            route_title = route.get_attribute('title')
            route_links.append(route_href)
            data.append({"route_link": route_href, "route_title": route_title})
        except TimeoutException as e:
            print("TimeoutException caught:", e)


finally:
    driver.quit()
# Print the collected data
for item in data:
    print(item)



{'route_link': 'https://www.redbus.in/bus-tickets/khammam-to-hyderabad', 'route_title': 'Khammam to Hyderabad'}
{'route_link': 'https://www.redbus.in/bus-tickets/hyderabad-to-vijayawada', 'route_title': 'Hyderabad to Vijayawada'}
{'route_link': 'https://www.redbus.in/bus-tickets/hyderabad-to-khammam', 'route_title': 'Hyderabad to Khammam'}
{'route_link': 'https://www.redbus.in/bus-tickets/hyderabad-to-srisailam', 'route_title': 'Hyderabad to Srisailam'}
{'route_link': 'https://www.redbus.in/bus-tickets/karimnagar-to-hyderabad', 'route_title': 'Karimnagar to Hyderabad'}
{'route_link': 'https://www.redbus.in/bus-tickets/hyderabad-to-nirmal', 'route_title': 'Hyderabad to Nirmal'}
{'route_link': 'https://www.redbus.in/bus-tickets/hyderabad-to-mancherial', 'route_title': 'Hyderabad to Mancherial'}
{'route_link': 'https://www.redbus.in/bus-tickets/hyderabad-to-adilabad', 'route_title': 'Hyderabad to Adilabad'}
{'route_link': 'https://www.redbus.in/bus-tickets/hyderabad-to-karimnagar', 'route

In [12]:
route_links

['https://www.redbus.in/bus-tickets/khammam-to-hyderabad',
 'https://www.redbus.in/bus-tickets/hyderabad-to-vijayawada',
 'https://www.redbus.in/bus-tickets/hyderabad-to-khammam',
 'https://www.redbus.in/bus-tickets/hyderabad-to-srisailam',
 'https://www.redbus.in/bus-tickets/karimnagar-to-hyderabad',
 'https://www.redbus.in/bus-tickets/hyderabad-to-nirmal',
 'https://www.redbus.in/bus-tickets/hyderabad-to-mancherial',
 'https://www.redbus.in/bus-tickets/hyderabad-to-adilabad',
 'https://www.redbus.in/bus-tickets/hyderabad-to-karimnagar',
 'https://www.redbus.in/bus-tickets/kothagudem-to-hyderabad']

In [19]:
route_detail=pd.DataFrame(data)
route_detail

,route_link,route_title
0,https://www.redbus.in/bus-tickets/khammam-to-h...,Khammam to Hyderabad
1,https://www.redbus.in/bus-tickets/hyderabad-to...,Hyderabad to Vijayawada
2,https://www.redbus.in/bus-tickets/hyderabad-to...,Hyderabad to Khammam
3,https://www.redbus.in/bus-tickets/hyderabad-to...,Hyderabad to Srisailam
4,https://www.redbus.in/bus-tickets/karimnagar-t...,Karimnagar to Hyderabad
5,https://www.redbus.in/bus-tickets/hyderabad-to...,Hyderabad to Nirmal
6,https://www.redbus.in/bus-tickets/hyderabad-to...,Hyderabad to Mancherial
7,https://www.redbus.in/bus-tickets/hyderabad-to...,Hyderabad to Adilabad
8,https://www.redbus.in/bus-tickets/hyderabad-to...,Hyderabad to Karimnagar
9,https://www.redbus.in/bus-tickets/kothagudem-t...,Kothagudem to Hyderabad


In [20]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

def scrape_bus_details(driver, route_links):
    details_list = []

    for link in route_links:
        driver.get(link)
        time.sleep(2)  # Allow the page to load initially
        
        # Get the body element for scrolling
        body = driver.find_element(By.TAG_NAME, "body")

        # Optional: Click on "View Buses" if needed
        try:
            view_buses = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, "//div[@class='button']"))
            )
            driver.execute_script("arguments[0].click();", view_buses)
        except:
            # Suppress error if the 'View Buses' button is not found
            pass

        # Scrolling logic
        old_page_source = driver.page_source
        while True:
            body.send_keys(Keys.PAGE_DOWN)
            time.sleep(2)  # Wait for new data to load
            
            new_page_source = driver.page_source
            if new_page_source == old_page_source:
                break
            old_page_source = new_page_source

        # Collect bus details
        wait = WebDriverWait(driver, 30)
        try:
            bus_containers = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "bus-item-details")))
            for container in bus_containers:
                try:
                    bus_name = container.find_element(By.XPATH, ".//div[@class='travels lh-24 f-bold d-color']").text
                except:
                    bus_name = "N/A"

                try:
                    bus_type = container.find_element(By.XPATH, ".//div[@class='bus-type f-12 m-top-16 l-color evBus']").text
                except:
                    bus_type = "N/A"

                try:
                    departing_time = container.find_element(By.XPATH, ".//div[@class='dp-time f-19 d-color f-bold']").text
                except:
                    departing_time = "N/A"

                try:
                    duration = container.find_element(By.XPATH, ".//div[@class='dur l-color lh-24']").text
                except:
                    duration = "N/A"

                try:
                    reaching_time = container.find_element(By.XPATH, ".//div[@class='bp-time f-19 d-color disp-Inline']").text
                except:
                    reaching_time = "N/A"

                try:
                    rating = container.find_element(By.CSS_SELECTOR, ".rating-sec .rating span").text
                except:
                    rating = "N/A"

                try:
                    price = container.find_element(By.XPATH, ".//div[contains(@class, 'fare d-block')]").text
                except:
                    price = "N/A"

                try:
                    seats_available = container.find_element(By.CLASS_NAME, "seat-left").text
                except:
                    seats_available = "N/A"

                details_list.append({
                    "bus_name": bus_name,
                    "bus_type": bus_type,
                    "departing_time": departing_time,
                    "duration": duration,
                    "reaching_time": reaching_time,
                    "rating": rating,
                    "price": price,
                    "seats_available": seats_available
                })

        except:
            # Suppress errors in finding bus containers
            pass

    return details_list

# Initialize the WebDriver
driver = webdriver.Chrome()

# Example list of route links
route_links = [
    'https://www.redbus.in/bus-tickets/khammam-to-hyderabad',
    'https://www.redbus.in/bus-tickets/hyderabad-to-vijayawada',
    'https://www.redbus.in/bus-tickets/hyderabad-to-khammam',
    'https://www.redbus.in/bus-tickets/hyderabad-to-srisailam',
    'https://www.redbus.in/bus-tickets/karimnagar-to-hyderabad',
    'https://www.redbus.in/bus-tickets/hyderabad-to-nirmal',
    'https://www.redbus.in/bus-tickets/hyderabad-to-mancherial',
    'https://www.redbus.in/bus-tickets/hyderabad-to-adilabad',
    'https://www.redbus.in/bus-tickets/hyderabad-to-karimnagar',
    'https://www.redbus.in/bus-tickets/kothagudem-to-hyderabad'
]

# Perform scraping
details_list = scrape_bus_details(driver, route_links)

# Print the collected details
print(details_list)

# Close the browser
driver.quit()


[{'bus_name': 'TSRTC - 8613', 'bus_type': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '17:20', 'duration': '04h 10m', 'reaching_time': '21:30', 'rating': '4.0', 'price': 'INR 320', 'seats_available': '26 Seats available'}, {'bus_name': 'TSRTC - 8675', 'bus_type': 'RAJDHANI (A.C. Semi Sleeper)', 'departing_time': '17:40', 'duration': '04h 00m', 'reaching_time': '21:40', 'rating': '4.1', 'price': 'INR 389', 'seats_available': '29 Seats available'}, {'bus_name': 'TSRTC - 64089', 'bus_type': 'RAJDHANI (A.C. Semi Sleeper)', 'departing_time': '18:00', 'duration': '04h 00m', 'reaching_time': '22:00', 'rating': '3.5', 'price': 'INR 389', 'seats_available': '35 Seats available'}, {'bus_name': 'TSRTC - 4172', 'bus_type': 'RAJADHANI AC (CONVERTED METRO LUXURY)', 'departing_time': '18:30', 'duration': '04h 00m', 'reaching_time': '22:30', 'rating': '4.1', 'price': 'INR 389', 'seats_available': '33 Seats available'}, {'bus_name': 'TSRTC - 64022', 'bus_type': 'RAJDHANI (A.C. Semi Slee

In [22]:
result1=pd.DataFrame(details_list)
result1

,bus_name,bus_type,departing_time,duration,reaching_time,rating,price,seats_available
0,TSRTC - 8613,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",17:20,04h 10m,21:30,4.0,INR 320,26 Seats available
1,TSRTC - 8675,RAJDHANI (A.C. Semi Sleeper),17:40,04h 00m,21:40,4.1,INR 389,29 Seats available
2,TSRTC - 64089,RAJDHANI (A.C. Semi Sleeper),18:00,04h 00m,22:00,3.5,INR 389,35 Seats available
3,TSRTC - 4172,RAJADHANI AC (CONVERTED METRO LUXURY),18:30,04h 00m,22:30,4.1,INR 389,33 Seats available
4,TSRTC - 64022,RAJDHANI (A.C. Semi Sleeper),19:00,02h 40m,21:40,2.9,INR 389,35 Seats available
...,...,...,...,...,...,...,...,...
134,BSR Tours And Travels,A/C Sleeper (2+1),23:30,06h 30m,06:00,4.6,780,7 Seats available
135,SAIRAJ TRAVELS,A/C Seater / Sleeper (2+1),23:00,06h 30m,05:30,4.6,INR 560,18 Seats available
136,BSR Tours And Travels,A/C Sleeper (2+1),23:15,06h 45m,06:00,4.7,780,14 Seats available
137,SAIRAJ TRAVELS,NON A/C Seater (2+2),23:05,06h 25m,05:30,4.4,INR 500,17 Seats available


In [30]:
result1=pd.concat([route_detail,result1],axis=1)
  # Forward fill


result1 = result1.ffill()
result1

,route_link,route_title,bus_name,bus_type,departing_time,duration,reaching_time,rating,price,seats_available
0,https://www.redbus.in/bus-tickets/khammam-to-h...,Khammam to Hyderabad,TSRTC - 8726,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",10:40,04h 30m,15:10,2.1,INR 320,31 Seats available
1,https://www.redbus.in/bus-tickets/hyderabad-to...,Hyderabad to Vijayawada,TSRTC - 8847,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",11:10,04h 20m,15:30,4.1,INR 320,31 Seats available
2,https://www.redbus.in/bus-tickets/hyderabad-to...,Hyderabad to Khammam,TSRTC - 8970,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",11:15,04h 00m,15:15,4.0,INR 320,31 Seats available
3,https://www.redbus.in/bus-tickets/hyderabad-to...,Hyderabad to Srisailam,TSRTC - 34339,RAJDHANI (A.C. Semi Sleeper),11:20,04h 10m,15:30,2.6,INR 389,29 Seats available
4,https://www.redbus.in/bus-tickets/karimnagar-t...,Karimnagar to Hyderabad,TSRTC - 8824,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",11:50,04h 40m,16:30,3.9,INR 320,31 Seats available
...,...,...,...,...,...,...,...,...,...,...
114,https://www.redbus.in/bus-tickets/kothagudem-t...,Kothagudem to Hyderabad,BSR Tours And Travels,A/C Sleeper (2+1),23:30,06h 30m,06:00,4.6,INR 800,19 Seats available
115,https://www.redbus.in/bus-tickets/kothagudem-t...,Kothagudem to Hyderabad,SAIRAJ TRAVELS,A/C Seater / Sleeper (2+1),23:00,06h 30m,05:30,4.6,INR 630,29 Seats available
116,https://www.redbus.in/bus-tickets/kothagudem-t...,Kothagudem to Hyderabad,BSR Tours And Travels,A/C Sleeper (2+1),23:15,06h 45m,06:00,4.7,INR 800,20 Seats available
117,https://www.redbus.in/bus-tickets/kothagudem-t...,Kothagudem to Hyderabad,SAIRAJ TRAVELS,NON A/C Seater (2+2),23:05,06h 25m,05:30,4.4,INR 500,32 Seats available


**JOINING APSRTC AND TSRTC**

In [23]:
result

,route_link,route_title,bus_name,bus_type,departing_time,duration,reaching_time,rating,price,seats_available
0,https://www.redbus.in/bus-tickets/vijayawada-t...,Vijayawada to Hyderabad,APSRTC - 35133,AMARAVATHI (VOLVO / SCANIA A.C Multi Axle),17:00,05h 50m,22:50,3.3,INR 720,33 Seats available
1,https://www.redbus.in/bus-tickets/hyderabad-to...,Hyderabad to Vijayawada,APSRTC - 35085,DOLPHIN CRUISE (VOLVO / SCANIA A.C Multi Axle),18:30,06h 00m,00:30,3.9,INR 720,38 Seats available
2,https://www.redbus.in/bus-tickets/kakinada-to-...,Kakinada to Visakhapatnam,APSRTC - 3613,Super Luxury (Non AC Seater 2+2 Push Back),19:00,07h 42m,02:42,3.1,INR 469,33 Seats available
3,https://www.redbus.in/bus-tickets/visakhapatna...,Visakhapatnam to Kakinada,APSRTC - 9316,Super Luxury (Non AC Seater 2+2 Push Back),20:20,07h 50m,04:10,4.1,INR 469,16 Seats available
4,https://www.redbus.in/bus-tickets/chittoor-and...,Chittoor (Andhra Pradesh) to Bangalore,APSRTC - 35189,DOLPHIN CRUISE (VOLVO / SCANIA A.C Multi Axle),20:30,04h 35m,01:05,3.8,INR 670,27 Seats available
...,...,...,...,...,...,...,...,...,...,...
140,https://www.redbus.in/bus-tickets/ongole-to-hy...,Ongole to Hyderabad,,,,,,,,
141,https://www.redbus.in/bus-tickets/ongole-to-hy...,Ongole to Hyderabad,,,,,,,,
142,https://www.redbus.in/bus-tickets/ongole-to-hy...,Ongole to Hyderabad,,,,,,,,
143,https://www.redbus.in/bus-tickets/ongole-to-hy...,Ongole to Hyderabad,,,,,,,,


In [24]:
result1

,bus_name,bus_type,departing_time,duration,reaching_time,rating,price,seats_available
0,TSRTC - 8613,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",17:20,04h 10m,21:30,4.0,INR 320,26 Seats available
1,TSRTC - 8675,RAJDHANI (A.C. Semi Sleeper),17:40,04h 00m,21:40,4.1,INR 389,29 Seats available
2,TSRTC - 64089,RAJDHANI (A.C. Semi Sleeper),18:00,04h 00m,22:00,3.5,INR 389,35 Seats available
3,TSRTC - 4172,RAJADHANI AC (CONVERTED METRO LUXURY),18:30,04h 00m,22:30,4.1,INR 389,33 Seats available
4,TSRTC - 64022,RAJDHANI (A.C. Semi Sleeper),19:00,02h 40m,21:40,2.9,INR 389,35 Seats available
...,...,...,...,...,...,...,...,...
134,BSR Tours And Travels,A/C Sleeper (2+1),23:30,06h 30m,06:00,4.6,780,7 Seats available
135,SAIRAJ TRAVELS,A/C Seater / Sleeper (2+1),23:00,06h 30m,05:30,4.6,INR 560,18 Seats available
136,BSR Tours And Travels,A/C Sleeper (2+1),23:15,06h 45m,06:00,4.7,780,14 Seats available
137,SAIRAJ TRAVELS,NON A/C Seater (2+2),23:05,06h 25m,05:30,4.4,INR 500,17 Seats available


In [25]:
ap_ts = pd.concat([result, result1], axis=0, ignore_index=True)

# Display the combined DataFrame
print(ap_ts)

                                            route_link  \
0    https://www.redbus.in/bus-tickets/vijayawada-t...   
1    https://www.redbus.in/bus-tickets/hyderabad-to...   
2    https://www.redbus.in/bus-tickets/kakinada-to-...   
3    https://www.redbus.in/bus-tickets/visakhapatna...   
4    https://www.redbus.in/bus-tickets/chittoor-and...   
..                                                 ...   
279                                                NaN   
280                                                NaN   
281                                                NaN   
282                                                NaN   
283                                                NaN   

                                route_title                    bus_name  \
0                   Vijayawada to Hyderabad              APSRTC - 35133   
1                   Hyderabad to Vijayawada              APSRTC - 35085   
2                 Kakinada to Visakhapatnam               APSRTC - 3613   
3  

**Joing ap and ts**

In [26]:
ap_ts=pd.DataFrame(ap_ts)
ap_ts

,route_link,route_title,bus_name,bus_type,departing_time,duration,reaching_time,rating,price,seats_available
0,https://www.redbus.in/bus-tickets/vijayawada-t...,Vijayawada to Hyderabad,APSRTC - 35133,AMARAVATHI (VOLVO / SCANIA A.C Multi Axle),17:00,05h 50m,22:50,3.3,INR 720,33 Seats available
1,https://www.redbus.in/bus-tickets/hyderabad-to...,Hyderabad to Vijayawada,APSRTC - 35085,DOLPHIN CRUISE (VOLVO / SCANIA A.C Multi Axle),18:30,06h 00m,00:30,3.9,INR 720,38 Seats available
2,https://www.redbus.in/bus-tickets/kakinada-to-...,Kakinada to Visakhapatnam,APSRTC - 3613,Super Luxury (Non AC Seater 2+2 Push Back),19:00,07h 42m,02:42,3.1,INR 469,33 Seats available
3,https://www.redbus.in/bus-tickets/visakhapatna...,Visakhapatnam to Kakinada,APSRTC - 9316,Super Luxury (Non AC Seater 2+2 Push Back),20:20,07h 50m,04:10,4.1,INR 469,16 Seats available
4,https://www.redbus.in/bus-tickets/chittoor-and...,Chittoor (Andhra Pradesh) to Bangalore,APSRTC - 35189,DOLPHIN CRUISE (VOLVO / SCANIA A.C Multi Axle),20:30,04h 35m,01:05,3.8,INR 670,27 Seats available
...,...,...,...,...,...,...,...,...,...,...
279,NaN,NaN,BSR Tours And Travels,A/C Sleeper (2+1),23:30,06h 30m,06:00,4.6,780,7 Seats available
280,NaN,NaN,SAIRAJ TRAVELS,A/C Seater / Sleeper (2+1),23:00,06h 30m,05:30,4.6,INR 560,18 Seats available
281,NaN,NaN,BSR Tours And Travels,A/C Sleeper (2+1),23:15,06h 45m,06:00,4.7,780,14 Seats available
282,NaN,NaN,SAIRAJ TRAVELS,NON A/C Seater (2+2),23:05,06h 25m,05:30,4.4,INR 500,17 Seats available


In [28]:
ap_ts=ap_ts.ffill()
ap_ts

,route_link,route_title,bus_name,bus_type,departing_time,duration,reaching_time,rating,price,seats_available
0,https://www.redbus.in/bus-tickets/vijayawada-t...,Vijayawada to Hyderabad,APSRTC - 35133,AMARAVATHI (VOLVO / SCANIA A.C Multi Axle),17:00,05h 50m,22:50,3.3,INR 720,33 Seats available
1,https://www.redbus.in/bus-tickets/hyderabad-to...,Hyderabad to Vijayawada,APSRTC - 35085,DOLPHIN CRUISE (VOLVO / SCANIA A.C Multi Axle),18:30,06h 00m,00:30,3.9,INR 720,38 Seats available
2,https://www.redbus.in/bus-tickets/kakinada-to-...,Kakinada to Visakhapatnam,APSRTC - 3613,Super Luxury (Non AC Seater 2+2 Push Back),19:00,07h 42m,02:42,3.1,INR 469,33 Seats available
3,https://www.redbus.in/bus-tickets/visakhapatna...,Visakhapatnam to Kakinada,APSRTC - 9316,Super Luxury (Non AC Seater 2+2 Push Back),20:20,07h 50m,04:10,4.1,INR 469,16 Seats available
4,https://www.redbus.in/bus-tickets/chittoor-and...,Chittoor (Andhra Pradesh) to Bangalore,APSRTC - 35189,DOLPHIN CRUISE (VOLVO / SCANIA A.C Multi Axle),20:30,04h 35m,01:05,3.8,INR 670,27 Seats available
...,...,...,...,...,...,...,...,...,...,...
279,https://www.redbus.in/bus-tickets/ongole-to-hy...,Ongole to Hyderabad,BSR Tours And Travels,A/C Sleeper (2+1),23:30,06h 30m,06:00,4.6,780,7 Seats available
280,https://www.redbus.in/bus-tickets/ongole-to-hy...,Ongole to Hyderabad,SAIRAJ TRAVELS,A/C Seater / Sleeper (2+1),23:00,06h 30m,05:30,4.6,INR 560,18 Seats available
281,https://www.redbus.in/bus-tickets/ongole-to-hy...,Ongole to Hyderabad,BSR Tours And Travels,A/C Sleeper (2+1),23:15,06h 45m,06:00,4.7,780,14 Seats available
282,https://www.redbus.in/bus-tickets/ongole-to-hy...,Ongole to Hyderabad,SAIRAJ TRAVELS,NON A/C Seater (2+2),23:05,06h 25m,05:30,4.4,INR 500,17 Seats available


**KSRTC**

In [29]:
#ksrtc
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
route_links=[]
data = []

# Initialize WebDriver
driver = webdriver.Chrome()

try:
    driver.get("https://www.redbus.in/online-booking/ksrtc-kerala/?utm_source=rtchometile")
    driver.maximize_window()
    
    # Initialize WebDriverWait
    wait = WebDriverWait(driver, 10)  # 10 seconds timeout
    
    # Wait for elements with the class name "route" to be present
    route_elements = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "route")))
    
    for route in route_elements:
        try:
            route_href = route.get_attribute('href')
            route_title = route.get_attribute('title')
            route_links.append(route_href)
            data.append({"route_link": route_href, "route_title": route_title})
        except TimeoutException as e:
            print("TimeoutException caught:", e)


finally:
    driver.quit()
# Print the collected data
for item in data:
    print(item)



{'route_link': 'https://www.redbus.in/bus-tickets/bangalore-to-kozhikode', 'route_title': 'Bangalore to Kozhikode'}
{'route_link': 'https://www.redbus.in/bus-tickets/kozhikode-to-ernakulam', 'route_title': 'Kozhikode to Ernakulam'}
{'route_link': 'https://www.redbus.in/bus-tickets/kozhikode-to-bangalore', 'route_title': 'Kozhikode to Bangalore'}
{'route_link': 'https://www.redbus.in/bus-tickets/ernakulam-to-kozhikode', 'route_title': 'Ernakulam to Kozhikode'}
{'route_link': 'https://www.redbus.in/bus-tickets/kozhikode-to-mysore', 'route_title': 'Kozhikode to Mysore'}
{'route_link': 'https://www.redbus.in/bus-tickets/kozhikode-to-thiruvananthapuram', 'route_title': 'Kozhikode to Thiruvananthapuram'}
{'route_link': 'https://www.redbus.in/bus-tickets/bangalore-to-kalpetta', 'route_title': 'Bangalore to Kalpetta (kerala)'}
{'route_link': 'https://www.redbus.in/bus-tickets/mysore-to-kozhikode', 'route_title': 'Mysore to Kozhikode'}
{'route_link': 'https://www.redbus.in/bus-tickets/kalpetta-

In [30]:
route_links

['https://www.redbus.in/bus-tickets/bangalore-to-kozhikode',
 'https://www.redbus.in/bus-tickets/kozhikode-to-ernakulam',
 'https://www.redbus.in/bus-tickets/kozhikode-to-bangalore',
 'https://www.redbus.in/bus-tickets/ernakulam-to-kozhikode',
 'https://www.redbus.in/bus-tickets/kozhikode-to-mysore',
 'https://www.redbus.in/bus-tickets/kozhikode-to-thiruvananthapuram',
 'https://www.redbus.in/bus-tickets/bangalore-to-kalpetta',
 'https://www.redbus.in/bus-tickets/mysore-to-kozhikode',
 'https://www.redbus.in/bus-tickets/kalpetta-to-bangalore',
 'https://www.redbus.in/bus-tickets/kozhikode-to-thrissur']

In [31]:
route_details=pd.DataFrame(data)
route_details

,route_link,route_title
0,https://www.redbus.in/bus-tickets/bangalore-to...,Bangalore to Kozhikode
1,https://www.redbus.in/bus-tickets/kozhikode-to...,Kozhikode to Ernakulam
2,https://www.redbus.in/bus-tickets/kozhikode-to...,Kozhikode to Bangalore
3,https://www.redbus.in/bus-tickets/ernakulam-to...,Ernakulam to Kozhikode
4,https://www.redbus.in/bus-tickets/kozhikode-to...,Kozhikode to Mysore
5,https://www.redbus.in/bus-tickets/kozhikode-to...,Kozhikode to Thiruvananthapuram
6,https://www.redbus.in/bus-tickets/bangalore-to...,Bangalore to Kalpetta (kerala)
7,https://www.redbus.in/bus-tickets/mysore-to-ko...,Mysore to Kozhikode
8,https://www.redbus.in/bus-tickets/kalpetta-to-...,Kalpetta (kerala) to Bangalore
9,https://www.redbus.in/bus-tickets/kozhikode-to...,Kozhikode to Thrissur


In [32]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

def scrape_bus_details(driver, route_links):
    details_list = []

    for link in route_links:
        driver.get(link)
        time.sleep(2)  # Allow the page to load initially
        
        # Get the body element for scrolling
        body = driver.find_element(By.TAG_NAME, "body")

        # Optional: Click on "View Buses" if needed
        try:
            view_buses = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, "//div[@class='button']"))
            )
            driver.execute_script("arguments[0].click();", view_buses)
        except Exception:
            # Suppress error if the 'View Buses' button is not found
            pass

        # Scrolling logic
        old_page_source = driver.page_source
        while True:
            body.send_keys(Keys.PAGE_DOWN)
            time.sleep(2)  # Wait for new data to load
            
            new_page_source = driver.page_source
            if new_page_source == old_page_source:
                break
            old_page_source = new_page_source

        # Collect bus details
        wait = WebDriverWait(driver, 30)
        try:
            bus_containers = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "bus-item-details")))
            for container in bus_containers:
                try:
                    bus_name = container.find_element(By.XPATH, ".//div[@class='travels lh-24 f-bold d-color']").text
                    bus_type = container.find_element(By.XPATH, ".//div[@class='bus-type f-12 m-top-16 l-color evBus']").text
                    departing_time = container.find_element(By.XPATH, ".//div[@class='dp-time f-19 d-color f-bold']").text
                    duration = container.find_element(By.XPATH, ".//div[@class='dur l-color lh-24']").text
                    reaching_time = container.find_element(By.XPATH, ".//div[@class='bp-time f-19 d-color disp-Inline']").text

                    try:
                        rating = container.find_element(By.CSS_SELECTOR, ".rating-sec .rating span").text
                    except Exception:
                        rating = "N/A"  # Default value if rating is not found

                    price = container.find_element(By.XPATH, ".//div[contains(@class, 'fare d-block')]").text
                    seats_available = container.find_element(By.CLASS_NAME, "seat-left").text

                    details_list.append({
                        "bus_name": bus_name,
                        "bus_type": bus_type,
                        "departing_time": departing_time,
                        "duration": duration,
                        "reaching_time": reaching_time,
                        "rating": rating,
                        "price": price,
                        "seats_available": seats_available
                    })

                except Exception:
                    # Suppress errors in extracting details from a container
                    pass
        except Exception:
            # Suppress errors in finding bus containers
            pass

    return details_list

# Initialize the WebDriver
driver = webdriver.Chrome()

# Example list of route links
route_links = ['https://www.redbus.in/bus-tickets/bangalore-to-kozhikode',
 'https://www.redbus.in/bus-tickets/kozhikode-to-ernakulam',
 'https://www.redbus.in/bus-tickets/kozhikode-to-bangalore',
 'https://www.redbus.in/bus-tickets/ernakulam-to-kozhikode',
 'https://www.redbus.in/bus-tickets/kozhikode-to-mysore',
 'https://www.redbus.in/bus-tickets/kozhikode-to-thiruvananthapuram',
 'https://www.redbus.in/bus-tickets/bangalore-to-kalpetta',
 'https://www.redbus.in/bus-tickets/mysore-to-kozhikode',
 'https://www.redbus.in/bus-tickets/kalpetta-to-bangalore',
 'https://www.redbus.in/bus-tickets/kozhikode-to-thrissur'
    
]

# Perform scraping
details_list = scrape_bus_details(driver, route_links)

# Print the collected details
print(details_list)

# Close the browser
driver.quit()


[{'bus_name': 'KSRTC (Kerala) - 51', 'bus_type': 'Swift Deluxe Non AC Air Bus (2+2)', 'departing_time': '19:04', 'duration': '09h 20m', 'reaching_time': '04:24', 'rating': '4.3', 'price': 'INR 640', 'seats_available': '22 Seats available'}, {'bus_name': 'KSRTC (Kerala) - 789', 'bus_type': 'Super Fast Non AC Seater (2+3)', 'departing_time': '20:00', 'duration': '08h 25m', 'reaching_time': '04:25', 'rating': '3.6', 'price': 'INR 473', 'seats_available': '23 Seats available'}, {'bus_name': 'KSRTC (Kerala) - 2133', 'bus_type': 'Swift Deluxe Non AC Air Bus (2+2)', 'departing_time': '22:12', 'duration': '09h 24m', 'reaching_time': '07:36', 'rating': '3.4', 'price': 'INR 640', 'seats_available': '11 Seats available'}, {'bus_name': 'KSRTC (Kerala) - 2063', 'bus_type': 'SWIFT-GARUDA A/C SEATER BUS', 'departing_time': '22:32', 'duration': '09h 03m', 'reaching_time': '07:35', 'rating': '3.8', 'price': 'INR 627', 'seats_available': '2 Seats available'}, {'bus_name': 'KSRTC (Kerala) - 574', 'bus_ty

In [33]:
df=pd.DataFrame(details_list)

In [34]:
df

,bus_name,bus_type,departing_time,duration,reaching_time,rating,price,seats_available
0,KSRTC (Kerala) - 51,Swift Deluxe Non AC Air Bus (2+2),19:04,09h 20m,04:24,4.3,INR 640,22 Seats available
1,KSRTC (Kerala) - 789,Super Fast Non AC Seater (2+3),20:00,08h 25m,04:25,3.6,INR 473,23 Seats available
2,KSRTC (Kerala) - 2133,Swift Deluxe Non AC Air Bus (2+2),22:12,09h 24m,07:36,3.4,INR 640,11 Seats available
3,KSRTC (Kerala) - 2063,SWIFT-GARUDA A/C SEATER BUS,22:32,09h 03m,07:35,3.8,INR 627,2 Seats available
4,KSRTC (Kerala) - 574,Super Express Non AC Seater Air Bus (2+2),23:00,10h 30m,09:30,3.9,INR 469,12 Seats available
...,...,...,...,...,...,...,...,...
141,EMERALD TRAVELS,Bharat Benz A/C Semi Sleeper (2+2),21:55,02h 35m,00:30,3.1,INR 762,37 Seats available
142,BLM Transports,A/C Sleeper (2+1),22:00,03h 00m,01:00,3.9,903,28 Seats available
143,Madhavi Travels,A/C Sleeper (2+1),22:20,02h 50m,01:10,4.0,902,16 Seats available
144,Kallada Travels (Suresh Kallada),A/C Sleeper (2+1),23:15,02h 26m,01:41,3.2,INR 850,1 Seat available


In [43]:
df=pd.concat([route_details,df],axis=1)
df

,route_link,route_title,bus_name,bus_type,departing_time,duration,reaching_time,rating,price,seats_available
0,https://www.redbus.in/bus-tickets/bangalore-to...,Bangalore to Kozhikode,KSRTC (Kerala) - 52,SWIFT-GARUDA A/C SEATER BUS,12:01,07h 54m,19:55,3.6,INR 945,30 Seats available
1,https://www.redbus.in/bus-tickets/kozhikode-to...,Kozhikode to Ernakulam,KSRTC (Kerala) - 1210,SWIFT-HYBRID NON AC SEATER CUM SLEEPER,12:55,08h 20m,21:15,3.8,INR 740,7 Seats available
2,https://www.redbus.in/bus-tickets/kozhikode-to...,Kozhikode to Bangalore,KSRTC (Kerala) - 129,SWIFT-GARUDA A/C SEATER BUS,14:15,08h 35m,22:50,4.2,INR 945,32 Seats available
3,https://www.redbus.in/bus-tickets/ernakulam-to...,Ernakulam to Kozhikode,KSRTC (Kerala) - 235,Swift Deluxe Non AC Air Bus (2+2),15:00,09h 01m,00:01,4.3,INR 801,17 Seats available
4,https://www.redbus.in/bus-tickets/kozhikode-to...,Kozhikode to Mysore,KSRTC (Kerala) - 370,AC MULTI AXLE,15:25,06h 55m,22:20,2.2,INR 1099,32 Seats available
...,...,...,...,...,...,...,...,...,...,...
192,NaN,NaN,EMERALD TRAVELS,A/C Semi Sleeper (2+2),22:00,01h 35m,23:35,3.1,INR 700,28 Seats available
193,NaN,NaN,BLM Transports,A/C Sleeper (2+1),22:00,03h 00m,01:00,3.9,1093,34 Seats available
194,NaN,NaN,Madhavi Travels,A/C Sleeper (2+1),22:20,02h 50m,01:10,4.0,999,15 Seats available
195,NaN,NaN,Kallada Travels (Suresh Kallada),A/C Sleeper (2+1),23:15,02h 26m,01:41,3.2,INR 850,23 Seats available


In [35]:
result2=df.ffill()
result2

,bus_name,bus_type,departing_time,duration,reaching_time,rating,price,seats_available
0,KSRTC (Kerala) - 51,Swift Deluxe Non AC Air Bus (2+2),19:04,09h 20m,04:24,4.3,INR 640,22 Seats available
1,KSRTC (Kerala) - 789,Super Fast Non AC Seater (2+3),20:00,08h 25m,04:25,3.6,INR 473,23 Seats available
2,KSRTC (Kerala) - 2133,Swift Deluxe Non AC Air Bus (2+2),22:12,09h 24m,07:36,3.4,INR 640,11 Seats available
3,KSRTC (Kerala) - 2063,SWIFT-GARUDA A/C SEATER BUS,22:32,09h 03m,07:35,3.8,INR 627,2 Seats available
4,KSRTC (Kerala) - 574,Super Express Non AC Seater Air Bus (2+2),23:00,10h 30m,09:30,3.9,INR 469,12 Seats available
...,...,...,...,...,...,...,...,...
141,EMERALD TRAVELS,Bharat Benz A/C Semi Sleeper (2+2),21:55,02h 35m,00:30,3.1,INR 762,37 Seats available
142,BLM Transports,A/C Sleeper (2+1),22:00,03h 00m,01:00,3.9,903,28 Seats available
143,Madhavi Travels,A/C Sleeper (2+1),22:20,02h 50m,01:10,4.0,902,16 Seats available
144,Kallada Travels (Suresh Kallada),A/C Sleeper (2+1),23:15,02h 26m,01:41,3.2,INR 850,1 Seat available


In [36]:
ap_ts_ks = pd.concat([result, result1,result2], axis=0, ignore_index=True)

# Display the combined DataFrame
print(ap_ts_ks)

                                            route_link  \
0    https://www.redbus.in/bus-tickets/vijayawada-t...   
1    https://www.redbus.in/bus-tickets/hyderabad-to...   
2    https://www.redbus.in/bus-tickets/kakinada-to-...   
3    https://www.redbus.in/bus-tickets/visakhapatna...   
4    https://www.redbus.in/bus-tickets/chittoor-and...   
..                                                 ...   
425                                                NaN   
426                                                NaN   
427                                                NaN   
428                                                NaN   
429                                                NaN   

                                route_title                          bus_name  \
0                   Vijayawada to Hyderabad                    APSRTC - 35133   
1                   Hyderabad to Vijayawada                    APSRTC - 35085   
2                 Kakinada to Visakhapatnam                 

In [37]:
ap_ts_ks=pd.DataFrame(ap_ts_ks)
ap_ts_ks

,route_link,route_title,bus_name,bus_type,departing_time,duration,reaching_time,rating,price,seats_available
0,https://www.redbus.in/bus-tickets/vijayawada-t...,Vijayawada to Hyderabad,APSRTC - 35133,AMARAVATHI (VOLVO / SCANIA A.C Multi Axle),17:00,05h 50m,22:50,3.3,INR 720,33 Seats available
1,https://www.redbus.in/bus-tickets/hyderabad-to...,Hyderabad to Vijayawada,APSRTC - 35085,DOLPHIN CRUISE (VOLVO / SCANIA A.C Multi Axle),18:30,06h 00m,00:30,3.9,INR 720,38 Seats available
2,https://www.redbus.in/bus-tickets/kakinada-to-...,Kakinada to Visakhapatnam,APSRTC - 3613,Super Luxury (Non AC Seater 2+2 Push Back),19:00,07h 42m,02:42,3.1,INR 469,33 Seats available
3,https://www.redbus.in/bus-tickets/visakhapatna...,Visakhapatnam to Kakinada,APSRTC - 9316,Super Luxury (Non AC Seater 2+2 Push Back),20:20,07h 50m,04:10,4.1,INR 469,16 Seats available
4,https://www.redbus.in/bus-tickets/chittoor-and...,Chittoor (Andhra Pradesh) to Bangalore,APSRTC - 35189,DOLPHIN CRUISE (VOLVO / SCANIA A.C Multi Axle),20:30,04h 35m,01:05,3.8,INR 670,27 Seats available
...,...,...,...,...,...,...,...,...,...,...
425,NaN,NaN,EMERALD TRAVELS,Bharat Benz A/C Semi Sleeper (2+2),21:55,02h 35m,00:30,3.1,INR 762,37 Seats available
426,NaN,NaN,BLM Transports,A/C Sleeper (2+1),22:00,03h 00m,01:00,3.9,903,28 Seats available
427,NaN,NaN,Madhavi Travels,A/C Sleeper (2+1),22:20,02h 50m,01:10,4.0,902,16 Seats available
428,NaN,NaN,Kallada Travels (Suresh Kallada),A/C Sleeper (2+1),23:15,02h 26m,01:41,3.2,INR 850,1 Seat available


In [38]:
ap_ts_ks=ap_ts_ks.ffill()
ap_ts_ks

,route_link,route_title,bus_name,bus_type,departing_time,duration,reaching_time,rating,price,seats_available
0,https://www.redbus.in/bus-tickets/vijayawada-t...,Vijayawada to Hyderabad,APSRTC - 35133,AMARAVATHI (VOLVO / SCANIA A.C Multi Axle),17:00,05h 50m,22:50,3.3,INR 720,33 Seats available
1,https://www.redbus.in/bus-tickets/hyderabad-to...,Hyderabad to Vijayawada,APSRTC - 35085,DOLPHIN CRUISE (VOLVO / SCANIA A.C Multi Axle),18:30,06h 00m,00:30,3.9,INR 720,38 Seats available
2,https://www.redbus.in/bus-tickets/kakinada-to-...,Kakinada to Visakhapatnam,APSRTC - 3613,Super Luxury (Non AC Seater 2+2 Push Back),19:00,07h 42m,02:42,3.1,INR 469,33 Seats available
3,https://www.redbus.in/bus-tickets/visakhapatna...,Visakhapatnam to Kakinada,APSRTC - 9316,Super Luxury (Non AC Seater 2+2 Push Back),20:20,07h 50m,04:10,4.1,INR 469,16 Seats available
4,https://www.redbus.in/bus-tickets/chittoor-and...,Chittoor (Andhra Pradesh) to Bangalore,APSRTC - 35189,DOLPHIN CRUISE (VOLVO / SCANIA A.C Multi Axle),20:30,04h 35m,01:05,3.8,INR 670,27 Seats available
...,...,...,...,...,...,...,...,...,...,...
425,https://www.redbus.in/bus-tickets/ongole-to-hy...,Ongole to Hyderabad,EMERALD TRAVELS,Bharat Benz A/C Semi Sleeper (2+2),21:55,02h 35m,00:30,3.1,INR 762,37 Seats available
426,https://www.redbus.in/bus-tickets/ongole-to-hy...,Ongole to Hyderabad,BLM Transports,A/C Sleeper (2+1),22:00,03h 00m,01:00,3.9,903,28 Seats available
427,https://www.redbus.in/bus-tickets/ongole-to-hy...,Ongole to Hyderabad,Madhavi Travels,A/C Sleeper (2+1),22:20,02h 50m,01:10,4.0,902,16 Seats available
428,https://www.redbus.in/bus-tickets/ongole-to-hy...,Ongole to Hyderabad,Kallada Travels (Suresh Kallada),A/C Sleeper (2+1),23:15,02h 26m,01:41,3.2,INR 850,1 Seat available


**SBSTC**

In [39]:
#ksrtc
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
route_links=[]
data = []

# Initialize WebDriver
driver = webdriver.Chrome()

try:
    driver.get("https://www.redbus.in/online-booking/south-bengal-state-transport-corporation-sbstc/?utm_source=rtchometile")
    driver.maximize_window()
    
    # Initialize WebDriverWait
    wait = WebDriverWait(driver, 10)  # 10 seconds timeout
    
    # Wait for elements with the class name "route" to be present
    route_elements = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "route")))
    
    for route in route_elements:
        try:
            route_href = route.get_attribute('href')
            route_title = route.get_attribute('title')
            route_links.append(route_href)
            data.append({"route_link": route_href, "route_title": route_title})
        except TimeoutException as e:
            print("TimeoutException caught:", e)


finally:
    driver.quit()
# Print the collected data
for item in data:
    print(item)



{'route_link': 'https://www.redbus.in/bus-tickets/burdwan-to-kolkata', 'route_title': 'Burdwan to Kolkata'}
{'route_link': 'https://www.redbus.in/bus-tickets/kolkata-to-burdwan', 'route_title': 'Kolkata to Burdwan'}
{'route_link': 'https://www.redbus.in/bus-tickets/durgapur-to-kolkata', 'route_title': 'Durgapur (West Bengal) to Kolkata'}
{'route_link': 'https://www.redbus.in/bus-tickets/kolkata-to-haldia', 'route_title': 'Kolkata to Haldia'}
{'route_link': 'https://www.redbus.in/bus-tickets/haldia-to-kolkata', 'route_title': 'Haldia to Kolkata'}
{'route_link': 'https://www.redbus.in/bus-tickets/kolkata-to-durgapur', 'route_title': 'Kolkata to Durgapur (West Bengal)'}
{'route_link': 'https://www.redbus.in/bus-tickets/kolkata-to-arambagh-west-bengal', 'route_title': 'Kolkata to Arambagh (West Bengal)'}
{'route_link': 'https://www.redbus.in/bus-tickets/digha-to-kolkata', 'route_title': 'Digha to Kolkata'}
{'route_link': 'https://www.redbus.in/bus-tickets/kolkata-to-digha', 'route_title': 

In [43]:
route_links

['https://www.redbus.in/bus-tickets/burdwan-to-kolkata',
 'https://www.redbus.in/bus-tickets/kolkata-to-burdwan',
 'https://www.redbus.in/bus-tickets/durgapur-to-kolkata',
 'https://www.redbus.in/bus-tickets/kolkata-to-haldia',
 'https://www.redbus.in/bus-tickets/haldia-to-kolkata',
 'https://www.redbus.in/bus-tickets/kolkata-to-durgapur',
 'https://www.redbus.in/bus-tickets/kolkata-to-arambagh-west-bengal',
 'https://www.redbus.in/bus-tickets/digha-to-kolkata',
 'https://www.redbus.in/bus-tickets/kolkata-to-digha',
 'https://www.redbus.in/bus-tickets/kolkata-to-bankura']

In [44]:
route_detail=pd.DataFrame(data)
route_detail

,route_link,route_title
0,https://www.redbus.in/bus-tickets/burdwan-to-k...,Burdwan to Kolkata
1,https://www.redbus.in/bus-tickets/kolkata-to-b...,Kolkata to Burdwan
2,https://www.redbus.in/bus-tickets/durgapur-to-...,Durgapur (West Bengal) to Kolkata
3,https://www.redbus.in/bus-tickets/kolkata-to-h...,Kolkata to Haldia
4,https://www.redbus.in/bus-tickets/haldia-to-ko...,Haldia to Kolkata
5,https://www.redbus.in/bus-tickets/kolkata-to-d...,Kolkata to Durgapur (West Bengal)
6,https://www.redbus.in/bus-tickets/kolkata-to-a...,Kolkata to Arambagh (West Bengal)
7,https://www.redbus.in/bus-tickets/digha-to-kol...,Digha to Kolkata
8,https://www.redbus.in/bus-tickets/kolkata-to-d...,Kolkata to Digha
9,https://www.redbus.in/bus-tickets/kolkata-to-b...,Kolkata to Bankura


In [42]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

def scrape_bus_details(driver, route_links):
    details_list = []

    for link in route_links:
        driver.get(link)
        time.sleep(2)  # Allow the page to load initially
        
        # Get the body element for scrolling
        body = driver.find_element(By.TAG_NAME, "body")

        # Optional: Click on "View Buses" if needed
        try:
            view_buses = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, "//div[@class='button']"))
            )
            driver.execute_script("arguments[0].click();", view_buses)
        except Exception:
            # Suppress error if the 'View Buses' button is not found
            pass

        # Scrolling logic
        old_page_source = driver.page_source
        while True:
            body.send_keys(Keys.PAGE_DOWN)
            time.sleep(2)  # Wait for new data to load
            
            new_page_source = driver.page_source
            if new_page_source == old_page_source:
                break
            old_page_source = new_page_source

        # Collect bus details
        wait = WebDriverWait(driver, 30)
        try:
            bus_containers = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "bus-item-details")))
            for container in bus_containers:
                try:
                    bus_name = container.find_element(By.XPATH, ".//div[@class='travels lh-24 f-bold d-color']").text
                    bus_type = container.find_element(By.XPATH, ".//div[@class='bus-type f-12 m-top-16 l-color evBus']").text
                    departing_time = container.find_element(By.XPATH, ".//div[@class='dp-time f-19 d-color f-bold']").text
                    duration = container.find_element(By.XPATH, ".//div[@class='dur l-color lh-24']").text
                    reaching_time = container.find_element(By.XPATH, ".//div[@class='bp-time f-19 d-color disp-Inline']").text

                    try:
                        rating = container.find_element(By.CSS_SELECTOR, ".rating-sec .rating span").text
                    except Exception:
                        rating = "N/A"  # Default value if rating is not found

                    price = container.find_element(By.XPATH, ".//div[contains(@class, 'fare d-block')]").text
                    seats_available = container.find_element(By.CLASS_NAME, "seat-left").text

                    details_list.append({
                        "bus_name": bus_name,
                        "bus_type": bus_type,
                        "departing_time": departing_time,
                        "duration": duration,
                        "reaching_time": reaching_time,
                        "rating": rating,
                        "price": price,
                        "seats_available": seats_available
                    })

                except Exception:
                    # Suppress errors in extracting details from a container
                    pass
        except Exception:
            # Suppress errors in finding bus containers
            pass

    return details_list

# Initialize the WebDriver
driver = webdriver.Chrome()

# Example list of route links
route_links = ['https://www.redbus.in/bus-tickets/burdwan-to-kolkata',
 'https://www.redbus.in/bus-tickets/kolkata-to-burdwan',
 'https://www.redbus.in/bus-tickets/durgapur-to-kolkata',
 'https://www.redbus.in/bus-tickets/kolkata-to-haldia',
 'https://www.redbus.in/bus-tickets/haldia-to-kolkata',
 'https://www.redbus.in/bus-tickets/kolkata-to-durgapur',
 'https://www.redbus.in/bus-tickets/kolkata-to-arambagh-west-bengal',
 'https://www.redbus.in/bus-tickets/digha-to-kolkata',
 'https://www.redbus.in/bus-tickets/kolkata-to-digha',
 'https://www.redbus.in/bus-tickets/kolkata-to-bankura']
    


# Perform scraping
details_list = scrape_bus_details(driver, route_links)

# Print the collected details
print(details_list)

# Close the browser
driver.quit()


[{'bus_name': 'SBSTC-BURDWAN - KARUNAMOYEE - 16:45 (BWN DEPOT) - 59', 'bus_type': 'Non AC Seater (2+3)', 'departing_time': '16:45', 'duration': '02h 30m', 'reaching_time': '19:15', 'rating': '3.7', 'price': 'INR 100', 'seats_available': '29 Seats available'}, {'bus_name': 'SBSTC-BURDWAN - KOLKATA - 17:00 (BWN DEPOT) - 41', 'bus_type': 'Non AC Seater (2+3)', 'departing_time': '17:00', 'duration': '02h 00m', 'reaching_time': '19:00', 'rating': '3.6', 'price': 'INR 95', 'seats_available': '38 Seats available'}, {'bus_name': 'SBSTC-BURDWAN - KARUNAMOYEE - 17:30 (BWN DEPOT) - 60', 'bus_type': 'Non AC Seater (2+3)', 'departing_time': '17:30', 'duration': '02h 30m', 'reaching_time': '20:00', 'rating': '3.9', 'price': 'INR 100', 'seats_available': '22 Seats available'}, {'bus_name': 'SBSTC-BURDWAN - KARUNAMOYEE - 18:30 (BWN DEPOT) - 61', 'bus_type': 'Non AC Seater (2+3)', 'departing_time': '18:30', 'duration': '02h 30m', 'reaching_time': '21:00', 'rating': '3.9', 'price': 'INR 100', 'seats_ava

In [45]:
df=pd.DataFrame(details_list)
df

,bus_name,bus_type,departing_time,duration,reaching_time,rating,price,seats_available
0,SBSTC-BURDWAN - KARUNAMOYEE - 16:45 (BWN DEPOT...,Non AC Seater (2+3),16:45,02h 30m,19:15,3.7,INR 100,29 Seats available
1,SBSTC-BURDWAN - KOLKATA - 17:00 (BWN DEPOT) - 41,Non AC Seater (2+3),17:00,02h 00m,19:00,3.6,INR 95,38 Seats available
2,SBSTC-BURDWAN - KARUNAMOYEE - 17:30 (BWN DEPOT...,Non AC Seater (2+3),17:30,02h 30m,20:00,3.9,INR 100,22 Seats available
3,SBSTC-BURDWAN - KARUNAMOYEE - 18:30 (BWN DEPOT...,Non AC Seater (2+3),18:30,02h 30m,21:00,3.9,INR 100,44 Seats available
4,Greenline,Volvo A/C Seater (2+2),16:00,02h 50m,18:50,5.0,INR 429,31 Seats available
...,...,...,...,...,...,...,...,...
69,City Express (Shinjini),AC Seater (2+3),23:50,05h 05m,04:55,3.4,INR 480,33 Seats available
70,Parameswar Travels,A/C Seater Push Back (2+3),23:45,05h 15m,05:00,3.0,285,45 Seats available
71,SBSTC-KARUNAMOYEE - BANKURA - VIA - DURGAPUR -...,Non AC Seater (2+3),16:45,05h 30m,22:15,4.2,INR 185,44 Seats available
72,City Queen Bus Service,Non AC Seater (2+2),20:50,05h 10m,02:00,3.0,INR 250,14 Seats available


In [46]:
result3=pd.concat([route_detail,df],axis=1)
result3

,route_link,route_title,bus_name,bus_type,departing_time,duration,reaching_time,rating,price,seats_available
0,https://www.redbus.in/bus-tickets/burdwan-to-k...,Burdwan to Kolkata,SBSTC-BURDWAN - KARUNAMOYEE - 16:45 (BWN DEPOT...,Non AC Seater (2+3),16:45,02h 30m,19:15,3.7,INR 100,29 Seats available
1,https://www.redbus.in/bus-tickets/kolkata-to-b...,Kolkata to Burdwan,SBSTC-BURDWAN - KOLKATA - 17:00 (BWN DEPOT) - 41,Non AC Seater (2+3),17:00,02h 00m,19:00,3.6,INR 95,38 Seats available
2,https://www.redbus.in/bus-tickets/durgapur-to-...,Durgapur (West Bengal) to Kolkata,SBSTC-BURDWAN - KARUNAMOYEE - 17:30 (BWN DEPOT...,Non AC Seater (2+3),17:30,02h 30m,20:00,3.9,INR 100,22 Seats available
3,https://www.redbus.in/bus-tickets/kolkata-to-h...,Kolkata to Haldia,SBSTC-BURDWAN - KARUNAMOYEE - 18:30 (BWN DEPOT...,Non AC Seater (2+3),18:30,02h 30m,21:00,3.9,INR 100,44 Seats available
4,https://www.redbus.in/bus-tickets/haldia-to-ko...,Haldia to Kolkata,Greenline,Volvo A/C Seater (2+2),16:00,02h 50m,18:50,5.0,INR 429,31 Seats available
...,...,...,...,...,...,...,...,...,...,...
69,NaN,NaN,City Express (Shinjini),AC Seater (2+3),23:50,05h 05m,04:55,3.4,INR 480,33 Seats available
70,NaN,NaN,Parameswar Travels,A/C Seater Push Back (2+3),23:45,05h 15m,05:00,3.0,285,45 Seats available
71,NaN,NaN,SBSTC-KARUNAMOYEE - BANKURA - VIA - DURGAPUR -...,Non AC Seater (2+3),16:45,05h 30m,22:15,4.2,INR 185,44 Seats available
72,NaN,NaN,City Queen Bus Service,Non AC Seater (2+2),20:50,05h 10m,02:00,3.0,INR 250,14 Seats available


In [47]:
result3=result3.ffill()
result3

,route_link,route_title,bus_name,bus_type,departing_time,duration,reaching_time,rating,price,seats_available
0,https://www.redbus.in/bus-tickets/burdwan-to-k...,Burdwan to Kolkata,SBSTC-BURDWAN - KARUNAMOYEE - 16:45 (BWN DEPOT...,Non AC Seater (2+3),16:45,02h 30m,19:15,3.7,INR 100,29 Seats available
1,https://www.redbus.in/bus-tickets/kolkata-to-b...,Kolkata to Burdwan,SBSTC-BURDWAN - KOLKATA - 17:00 (BWN DEPOT) - 41,Non AC Seater (2+3),17:00,02h 00m,19:00,3.6,INR 95,38 Seats available
2,https://www.redbus.in/bus-tickets/durgapur-to-...,Durgapur (West Bengal) to Kolkata,SBSTC-BURDWAN - KARUNAMOYEE - 17:30 (BWN DEPOT...,Non AC Seater (2+3),17:30,02h 30m,20:00,3.9,INR 100,22 Seats available
3,https://www.redbus.in/bus-tickets/kolkata-to-h...,Kolkata to Haldia,SBSTC-BURDWAN - KARUNAMOYEE - 18:30 (BWN DEPOT...,Non AC Seater (2+3),18:30,02h 30m,21:00,3.9,INR 100,44 Seats available
4,https://www.redbus.in/bus-tickets/haldia-to-ko...,Haldia to Kolkata,Greenline,Volvo A/C Seater (2+2),16:00,02h 50m,18:50,5.0,INR 429,31 Seats available
...,...,...,...,...,...,...,...,...,...,...
69,https://www.redbus.in/bus-tickets/kolkata-to-b...,Kolkata to Bankura,City Express (Shinjini),AC Seater (2+3),23:50,05h 05m,04:55,3.4,INR 480,33 Seats available
70,https://www.redbus.in/bus-tickets/kolkata-to-b...,Kolkata to Bankura,Parameswar Travels,A/C Seater Push Back (2+3),23:45,05h 15m,05:00,3.0,285,45 Seats available
71,https://www.redbus.in/bus-tickets/kolkata-to-b...,Kolkata to Bankura,SBSTC-KARUNAMOYEE - BANKURA - VIA - DURGAPUR -...,Non AC Seater (2+3),16:45,05h 30m,22:15,4.2,INR 185,44 Seats available
72,https://www.redbus.in/bus-tickets/kolkata-to-b...,Kolkata to Bankura,City Queen Bus Service,Non AC Seater (2+2),20:50,05h 10m,02:00,3.0,INR 250,14 Seats available


In [48]:
ap_ts_ks_sb = pd.concat([result, result1,result2,result3], axis=0, ignore_index=True)

# Display the combined DataFrame
print(ap_ts_ks_sb)

                                            route_link  \
0    https://www.redbus.in/bus-tickets/vijayawada-t...   
1    https://www.redbus.in/bus-tickets/hyderabad-to...   
2    https://www.redbus.in/bus-tickets/kakinada-to-...   
3    https://www.redbus.in/bus-tickets/visakhapatna...   
4    https://www.redbus.in/bus-tickets/chittoor-and...   
..                                                 ...   
499  https://www.redbus.in/bus-tickets/kolkata-to-b...   
500  https://www.redbus.in/bus-tickets/kolkata-to-b...   
501  https://www.redbus.in/bus-tickets/kolkata-to-b...   
502  https://www.redbus.in/bus-tickets/kolkata-to-b...   
503  https://www.redbus.in/bus-tickets/kolkata-to-b...   

                                route_title  \
0                   Vijayawada to Hyderabad   
1                   Hyderabad to Vijayawada   
2                 Kakinada to Visakhapatnam   
3                 Visakhapatnam to Kakinada   
4    Chittoor (Andhra Pradesh) to Bangalore   
..                   

**JOINING AP TS KS SB**

In [49]:
ap_ts_ks_sb=pd.DataFrame(ap_ts_ks_sb)
ap_ts_ks_sb

,route_link,route_title,bus_name,bus_type,departing_time,duration,reaching_time,rating,price,seats_available
0,https://www.redbus.in/bus-tickets/vijayawada-t...,Vijayawada to Hyderabad,APSRTC - 35133,AMARAVATHI (VOLVO / SCANIA A.C Multi Axle),17:00,05h 50m,22:50,3.3,INR 720,33 Seats available
1,https://www.redbus.in/bus-tickets/hyderabad-to...,Hyderabad to Vijayawada,APSRTC - 35085,DOLPHIN CRUISE (VOLVO / SCANIA A.C Multi Axle),18:30,06h 00m,00:30,3.9,INR 720,38 Seats available
2,https://www.redbus.in/bus-tickets/kakinada-to-...,Kakinada to Visakhapatnam,APSRTC - 3613,Super Luxury (Non AC Seater 2+2 Push Back),19:00,07h 42m,02:42,3.1,INR 469,33 Seats available
3,https://www.redbus.in/bus-tickets/visakhapatna...,Visakhapatnam to Kakinada,APSRTC - 9316,Super Luxury (Non AC Seater 2+2 Push Back),20:20,07h 50m,04:10,4.1,INR 469,16 Seats available
4,https://www.redbus.in/bus-tickets/chittoor-and...,Chittoor (Andhra Pradesh) to Bangalore,APSRTC - 35189,DOLPHIN CRUISE (VOLVO / SCANIA A.C Multi Axle),20:30,04h 35m,01:05,3.8,INR 670,27 Seats available
...,...,...,...,...,...,...,...,...,...,...
499,https://www.redbus.in/bus-tickets/kolkata-to-b...,Kolkata to Bankura,City Express (Shinjini),AC Seater (2+3),23:50,05h 05m,04:55,3.4,INR 480,33 Seats available
500,https://www.redbus.in/bus-tickets/kolkata-to-b...,Kolkata to Bankura,Parameswar Travels,A/C Seater Push Back (2+3),23:45,05h 15m,05:00,3.0,285,45 Seats available
501,https://www.redbus.in/bus-tickets/kolkata-to-b...,Kolkata to Bankura,SBSTC-KARUNAMOYEE - BANKURA - VIA - DURGAPUR -...,Non AC Seater (2+3),16:45,05h 30m,22:15,4.2,INR 185,44 Seats available
502,https://www.redbus.in/bus-tickets/kolkata-to-b...,Kolkata to Bankura,City Queen Bus Service,Non AC Seater (2+2),20:50,05h 10m,02:00,3.0,INR 250,14 Seats available


**WBSTC**

In [50]:
#ksrtc
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
route_links=[]
data = []

# Initialize WebDriver
driver = webdriver.Chrome()

try:
    driver.get("https://www.redbus.in/online-booking/west-bengal-transport-corporation?utm_source=rtchometile")
    driver.maximize_window()
    
    # Initialize WebDriverWait
    wait = WebDriverWait(driver, 10)  # 10 seconds timeout
    
    # Wait for elements with the class name "route" to be present
    route_elements = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "route")))
    
    for route in route_elements:
        try:
            route_href = route.get_attribute('href')
            route_title = route.get_attribute('title')
            route_links.append(route_href)
            data.append({"route_link": route_href, "route_title": route_title})
        except TimeoutException as e:
            print("TimeoutException caught:", e)


finally:
    driver.quit()
# Print the collected data
for item in data:
    print(item)



{'route_link': 'https://www.redbus.in/bus-tickets/kolkata-to-digha', 'route_title': 'Kolkata to Digha'}
{'route_link': 'https://www.redbus.in/bus-tickets/digha-to-kolkata', 'route_title': 'Digha to Kolkata'}
{'route_link': 'https://www.redbus.in/bus-tickets/mandarmani-to-kolkata', 'route_title': 'Mandarmani to Kolkata'}
{'route_link': 'https://www.redbus.in/bus-tickets/kolkata-to-mandarmani', 'route_title': 'Kolkata to Mandarmani'}
{'route_link': 'https://www.redbus.in/bus-tickets/kolkata-to-bakkhali', 'route_title': 'Kolkata to Bakkhali'}


In [51]:
route_details=pd.DataFrame(data)
route_details

,route_link,route_title
0,https://www.redbus.in/bus-tickets/kolkata-to-d...,Kolkata to Digha
1,https://www.redbus.in/bus-tickets/digha-to-kol...,Digha to Kolkata
2,https://www.redbus.in/bus-tickets/mandarmani-t...,Mandarmani to Kolkata
3,https://www.redbus.in/bus-tickets/kolkata-to-m...,Kolkata to Mandarmani
4,https://www.redbus.in/bus-tickets/kolkata-to-b...,Kolkata to Bakkhali


In [52]:
route_links

['https://www.redbus.in/bus-tickets/kolkata-to-digha',
 'https://www.redbus.in/bus-tickets/digha-to-kolkata',
 'https://www.redbus.in/bus-tickets/mandarmani-to-kolkata',
 'https://www.redbus.in/bus-tickets/kolkata-to-mandarmani',
 'https://www.redbus.in/bus-tickets/kolkata-to-bakkhali']

In [53]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

def scrape_bus_details(driver, route_links):
    details_list = []

    for link in route_links:
        driver.get(link)
        time.sleep(2)  # Allow the page to load initially
        
        # Get the body element for scrolling
        body = driver.find_element(By.TAG_NAME, "body")

        # Optional: Click on "View Buses" if needed
        try:
            view_buses = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, "//div[@class='button']"))
            )
            driver.execute_script("arguments[0].click();", view_buses)
        except Exception:
            # Suppress error if the 'View Buses' button is not found
            pass

        # Scrolling logic
        old_page_source = driver.page_source
        while True:
            body.send_keys(Keys.PAGE_DOWN)
            time.sleep(2)  # Wait for new data to load
            
            new_page_source = driver.page_source
            if new_page_source == old_page_source:
                break
            old_page_source = new_page_source

        # Collect bus details
        wait = WebDriverWait(driver, 30)
        try:
            bus_containers = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "bus-item-details")))
            for container in bus_containers:
                try:
                    bus_name = container.find_element(By.XPATH, ".//div[@class='travels lh-24 f-bold d-color']").text
                    bus_type = container.find_element(By.XPATH, ".//div[@class='bus-type f-12 m-top-16 l-color evBus']").text
                    departing_time = container.find_element(By.XPATH, ".//div[@class='dp-time f-19 d-color f-bold']").text
                    duration = container.find_element(By.XPATH, ".//div[@class='dur l-color lh-24']").text
                    reaching_time = container.find_element(By.XPATH, ".//div[@class='bp-time f-19 d-color disp-Inline']").text

                    try:
                        rating = container.find_element(By.CSS_SELECTOR, ".rating-sec .rating span").text
                    except Exception:
                        rating = "N/A"  # Default value if rating is not found

                    price = container.find_element(By.XPATH, ".//div[contains(@class, 'fare d-block')]").text
                    seats_available = container.find_element(By.CLASS_NAME, "seat-left").text

                    details_list.append({
                        "bus_name": bus_name,
                        "bus_type": bus_type,
                        "departing_time": departing_time,
                        "duration": duration,
                        "reaching_time": reaching_time,
                        "rating": rating,
                        "price": price,
                        "seats_available": seats_available
                    })

                except Exception:
                    # Suppress errors in extracting details from a container
                    pass
        except Exception:
            # Suppress errors in finding bus containers
            pass

    return details_list

# Initialize the WebDriver
driver = webdriver.Chrome()

# Example list of route links
route_links = ['https://www.redbus.in/bus-tickets/kolkata-to-digha',
 'https://www.redbus.in/bus-tickets/digha-to-kolkata',
 'https://www.redbus.in/bus-tickets/mandarmani-to-kolkata',
 'https://www.redbus.in/bus-tickets/kolkata-to-mandarmani',
 'https://www.redbus.in/bus-tickets/kolkata-to-bakkhali']
    


# Perform scraping
details_list = scrape_bus_details(driver, route_links)

# Print the collected details
print(details_list)

# Close the browser
driver.quit()


[{'bus_name': 'SBSTC-GARIA - DIGHA - 22:30 (DIGHA DEPOT) - 3745', 'bus_type': 'Non AC Seater (2+3)', 'departing_time': '23:45', 'duration': '04h 30m', 'reaching_time': '04:15', 'rating': '4.2', 'price': 'INR 155', 'seats_available': '5 Seats available'}, {'bus_name': 'Express Line', 'bus_type': 'Volvo 9600 A/C Seater (2+2)', 'departing_time': '23:45', 'duration': '05h 00m', 'reaching_time': '04:45', 'rating': '4.5', 'price': '450', 'seats_available': '27 Seats available'}, {'bus_name': 'Shyamoli Paribahan Pvt Ltd', 'bus_type': 'VE A/C Seater (2+2)', 'departing_time': '23:30', 'duration': '05h 00m', 'reaching_time': '04:30', 'rating': '4.4', 'price': '450', 'seats_available': '15 Seats available'}, {'bus_name': 'Maa Chandi Travels', 'bus_type': 'A/C Seater / Sleeper (2+2)', 'departing_time': '22:00', 'duration': '07h 30m', 'reaching_time': '05:30', 'rating': '4.5', 'price': '370', 'seats_available': '18 Seats available'}, {'bus_name': 'Shyamoli Paribahan Pvt Ltd', 'bus_type': 'Volvo A/C

In [54]:
result4=pd.DataFrame(details_list)
result4

,bus_name,bus_type,departing_time,duration,reaching_time,rating,price,seats_available
0,SBSTC-GARIA - DIGHA - 22:30 (DIGHA DEPOT) - 3745,Non AC Seater (2+3),23:45,04h 30m,04:15,4.2,INR 155,5 Seats available
1,Express Line,Volvo 9600 A/C Seater (2+2),23:45,05h 00m,04:45,4.5,450,27 Seats available
2,Shyamoli Paribahan Pvt Ltd,VE A/C Seater (2+2),23:30,05h 00m,04:30,4.4,450,15 Seats available
3,Maa Chandi Travels,A/C Seater / Sleeper (2+2),22:00,07h 30m,05:30,4.5,370,18 Seats available
4,Shyamoli Paribahan Pvt Ltd,Volvo A/C Seater (2+2),23:45,05h 00m,04:45,4.5,INR 500,28 Seats available
5,Sagufta Travels(Maa Chandi),A/C Seater / Sleeper (2+2),23:57,05h 13m,05:10,4.2,394,30 Seats available
6,Maa Chandi Travels,A/C Seater / Sleeper (2+2),23:57,05h 13m,05:10,4.4,370,21 Seats available
7,Parameswar Travels,A/C Seater Push Back (2+2),23:50,04h 55m,04:45,3.7,285,15 Seats available
8,City Express (Shinjini),AC Seater (2+3),23:50,05h 05m,04:55,3.4,INR 480,33 Seats available
9,Sagufta Travels (Bhorpet Travels),AC Seater (2+2),23:55,04h 40m,04:35,3.9,394,24 Seats available


In [55]:
result4=pd.concat([route_details,result4],axis=1)
result4

,route_link,route_title,bus_name,bus_type,departing_time,duration,reaching_time,rating,price,seats_available
0,https://www.redbus.in/bus-tickets/kolkata-to-d...,Kolkata to Digha,SBSTC-GARIA - DIGHA - 22:30 (DIGHA DEPOT) - 3745,Non AC Seater (2+3),23:45,04h 30m,04:15,4.2,INR 155,5 Seats available
1,https://www.redbus.in/bus-tickets/digha-to-kol...,Digha to Kolkata,Express Line,Volvo 9600 A/C Seater (2+2),23:45,05h 00m,04:45,4.5,450,27 Seats available
2,https://www.redbus.in/bus-tickets/mandarmani-t...,Mandarmani to Kolkata,Shyamoli Paribahan Pvt Ltd,VE A/C Seater (2+2),23:30,05h 00m,04:30,4.4,450,15 Seats available
3,https://www.redbus.in/bus-tickets/kolkata-to-m...,Kolkata to Mandarmani,Maa Chandi Travels,A/C Seater / Sleeper (2+2),22:00,07h 30m,05:30,4.5,370,18 Seats available
4,https://www.redbus.in/bus-tickets/kolkata-to-b...,Kolkata to Bakkhali,Shyamoli Paribahan Pvt Ltd,Volvo A/C Seater (2+2),23:45,05h 00m,04:45,4.5,INR 500,28 Seats available
5,NaN,NaN,Sagufta Travels(Maa Chandi),A/C Seater / Sleeper (2+2),23:57,05h 13m,05:10,4.2,394,30 Seats available
6,NaN,NaN,Maa Chandi Travels,A/C Seater / Sleeper (2+2),23:57,05h 13m,05:10,4.4,370,21 Seats available
7,NaN,NaN,Parameswar Travels,A/C Seater Push Back (2+2),23:50,04h 55m,04:45,3.7,285,15 Seats available
8,NaN,NaN,City Express (Shinjini),AC Seater (2+3),23:50,05h 05m,04:55,3.4,INR 480,33 Seats available
9,NaN,NaN,Sagufta Travels (Bhorpet Travels),AC Seater (2+2),23:55,04h 40m,04:35,3.9,394,24 Seats available


In [56]:
result4=result4.ffill()
result4

,route_link,route_title,bus_name,bus_type,departing_time,duration,reaching_time,rating,price,seats_available
0,https://www.redbus.in/bus-tickets/kolkata-to-d...,Kolkata to Digha,SBSTC-GARIA - DIGHA - 22:30 (DIGHA DEPOT) - 3745,Non AC Seater (2+3),23:45,04h 30m,04:15,4.2,INR 155,5 Seats available
1,https://www.redbus.in/bus-tickets/digha-to-kol...,Digha to Kolkata,Express Line,Volvo 9600 A/C Seater (2+2),23:45,05h 00m,04:45,4.5,450,27 Seats available
2,https://www.redbus.in/bus-tickets/mandarmani-t...,Mandarmani to Kolkata,Shyamoli Paribahan Pvt Ltd,VE A/C Seater (2+2),23:30,05h 00m,04:30,4.4,450,15 Seats available
3,https://www.redbus.in/bus-tickets/kolkata-to-m...,Kolkata to Mandarmani,Maa Chandi Travels,A/C Seater / Sleeper (2+2),22:00,07h 30m,05:30,4.5,370,18 Seats available
4,https://www.redbus.in/bus-tickets/kolkata-to-b...,Kolkata to Bakkhali,Shyamoli Paribahan Pvt Ltd,Volvo A/C Seater (2+2),23:45,05h 00m,04:45,4.5,INR 500,28 Seats available
5,https://www.redbus.in/bus-tickets/kolkata-to-b...,Kolkata to Bakkhali,Sagufta Travels(Maa Chandi),A/C Seater / Sleeper (2+2),23:57,05h 13m,05:10,4.2,394,30 Seats available
6,https://www.redbus.in/bus-tickets/kolkata-to-b...,Kolkata to Bakkhali,Maa Chandi Travels,A/C Seater / Sleeper (2+2),23:57,05h 13m,05:10,4.4,370,21 Seats available
7,https://www.redbus.in/bus-tickets/kolkata-to-b...,Kolkata to Bakkhali,Parameswar Travels,A/C Seater Push Back (2+2),23:50,04h 55m,04:45,3.7,285,15 Seats available
8,https://www.redbus.in/bus-tickets/kolkata-to-b...,Kolkata to Bakkhali,City Express (Shinjini),AC Seater (2+3),23:50,05h 05m,04:55,3.4,INR 480,33 Seats available
9,https://www.redbus.in/bus-tickets/kolkata-to-b...,Kolkata to Bakkhali,Sagufta Travels (Bhorpet Travels),AC Seater (2+2),23:55,04h 40m,04:35,3.9,394,24 Seats available


In [57]:
ap_ts_ks_sb_wb = pd.concat([result, result1,result2,result3,result4], axis=0, ignore_index=True)

# Display the combined DataFrame
print(ap_ts_ks_sb_wb)

                                            route_link  \
0    https://www.redbus.in/bus-tickets/vijayawada-t...   
1    https://www.redbus.in/bus-tickets/hyderabad-to...   
2    https://www.redbus.in/bus-tickets/kakinada-to-...   
3    https://www.redbus.in/bus-tickets/visakhapatna...   
4    https://www.redbus.in/bus-tickets/chittoor-and...   
..                                                 ...   
541  https://www.redbus.in/bus-tickets/kolkata-to-b...   
542  https://www.redbus.in/bus-tickets/kolkata-to-b...   
543  https://www.redbus.in/bus-tickets/kolkata-to-b...   
544  https://www.redbus.in/bus-tickets/kolkata-to-b...   
545  https://www.redbus.in/bus-tickets/kolkata-to-b...   

                                route_title                       bus_name  \
0                   Vijayawada to Hyderabad                 APSRTC - 35133   
1                   Hyderabad to Vijayawada                 APSRTC - 35085   
2                 Kakinada to Visakhapatnam                  APSRTC -

In [58]:
ap_ts_ks_sb_wb=pd.DataFrame(ap_ts_ks_sb_wb)
ap_ts_ks_sb_wb

,route_link,route_title,bus_name,bus_type,departing_time,duration,reaching_time,rating,price,seats_available
0,https://www.redbus.in/bus-tickets/vijayawada-t...,Vijayawada to Hyderabad,APSRTC - 35133,AMARAVATHI (VOLVO / SCANIA A.C Multi Axle),17:00,05h 50m,22:50,3.3,INR 720,33 Seats available
1,https://www.redbus.in/bus-tickets/hyderabad-to...,Hyderabad to Vijayawada,APSRTC - 35085,DOLPHIN CRUISE (VOLVO / SCANIA A.C Multi Axle),18:30,06h 00m,00:30,3.9,INR 720,38 Seats available
2,https://www.redbus.in/bus-tickets/kakinada-to-...,Kakinada to Visakhapatnam,APSRTC - 3613,Super Luxury (Non AC Seater 2+2 Push Back),19:00,07h 42m,02:42,3.1,INR 469,33 Seats available
3,https://www.redbus.in/bus-tickets/visakhapatna...,Visakhapatnam to Kakinada,APSRTC - 9316,Super Luxury (Non AC Seater 2+2 Push Back),20:20,07h 50m,04:10,4.1,INR 469,16 Seats available
4,https://www.redbus.in/bus-tickets/chittoor-and...,Chittoor (Andhra Pradesh) to Bangalore,APSRTC - 35189,DOLPHIN CRUISE (VOLVO / SCANIA A.C Multi Axle),20:30,04h 35m,01:05,3.8,INR 670,27 Seats available
...,...,...,...,...,...,...,...,...,...,...
541,https://www.redbus.in/bus-tickets/kolkata-to-b...,Kolkata to Bakkhali,Samanta Travels,A/C Seater (2+3),18:10,04h 50m,23:00,3.4,352,16 Seats available
542,https://www.redbus.in/bus-tickets/kolkata-to-b...,Kolkata to Bakkhali,Maa Chandi Travels,A/C Seater / Sleeper (2+2),22:00,07h 10m,05:10,2.5,370,22 Seats available
543,https://www.redbus.in/bus-tickets/kolkata-to-b...,Kolkata to Bakkhali,Snemita Paribahan(Shinjini),A/C Seater (2+3),23:50,04h 10m,04:00,2.9,INR 480,16 Seats available
544,https://www.redbus.in/bus-tickets/kolkata-to-b...,Kolkata to Bakkhali,Parameswar Travels,A/C Seater Push Back (2+3),23:55,04h 35m,04:30,2.5,340,49 Seats available


In [59]:
#ksrtc
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
route_links=[]
data = []

# Initialize WebDriver
driver = webdriver.Chrome()

try:
    driver.get("https://www.redbus.in/online-booking/bihar-state-road-transport-corporation-bsrtc/?utm_source=rtchometile")
    driver.maximize_window()
    
    # Initialize WebDriverWait
    wait = WebDriverWait(driver, 10)  # 10 seconds timeout
    
    # Wait for elements with the class name "route" to be present
    route_elements = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "route")))
    
    for route in route_elements:
        try:
            route_href = route.get_attribute('href')
            route_title = route.get_attribute('title')
            route_links.append(route_href)
            data.append({"route_link": route_href, "route_title": route_title})
        except TimeoutException as e:
            print("TimeoutException caught:", e)


finally:
    driver.quit()
# Print the collected data
for item in data:
    print(item)



{'route_link': 'https://www.redbus.in/bus-tickets/patna-to-bettiah', 'route_title': 'Patna (Bihar) to Bettiah'}
{'route_link': 'https://www.redbus.in/bus-tickets/gopalganj-to-delhi', 'route_title': 'Gopalganj (Bihar) to Delhi'}
{'route_link': 'https://www.redbus.in/bus-tickets/patna-to-motihari', 'route_title': 'Patna (Bihar) to Motihari'}
{'route_link': 'https://www.redbus.in/bus-tickets/delhi-to-motihari', 'route_title': 'Delhi to Motihari'}
{'route_link': 'https://www.redbus.in/bus-tickets/bettiah-to-patna', 'route_title': 'Bettiah to Patna (Bihar)'}
{'route_link': 'https://www.redbus.in/bus-tickets/motihari-to-delhi', 'route_title': 'Motihari to Delhi'}
{'route_link': 'https://www.redbus.in/bus-tickets/patna-to-balmiki-nagar-bihar', 'route_title': 'Patna (Bihar) to Balmiki Nagar (bihar)'}
{'route_link': 'https://www.redbus.in/bus-tickets/balmiki-nagar-bihar-to-patna', 'route_title': 'Balmiki Nagar (bihar) to Patna (Bihar)'}
{'route_link': 'https://www.redbus.in/bus-tickets/patna-to

In [60]:
route_details=pd.DataFrame(data)
route_details

,route_link,route_title
0,https://www.redbus.in/bus-tickets/patna-to-bet...,Patna (Bihar) to Bettiah
1,https://www.redbus.in/bus-tickets/gopalganj-to...,Gopalganj (Bihar) to Delhi
2,https://www.redbus.in/bus-tickets/patna-to-mot...,Patna (Bihar) to Motihari
3,https://www.redbus.in/bus-tickets/delhi-to-mot...,Delhi to Motihari
4,https://www.redbus.in/bus-tickets/bettiah-to-p...,Bettiah to Patna (Bihar)
5,https://www.redbus.in/bus-tickets/motihari-to-...,Motihari to Delhi
6,https://www.redbus.in/bus-tickets/patna-to-bal...,Patna (Bihar) to Balmiki Nagar (bihar)
7,https://www.redbus.in/bus-tickets/balmiki-naga...,Balmiki Nagar (bihar) to Patna (Bihar)
8,https://www.redbus.in/bus-tickets/patna-to-kat...,Patna (Bihar) to Kathmandu
9,https://www.redbus.in/bus-tickets/patna-to-kat...,Patna (Bihar) to Katihar


In [61]:
route_links

['https://www.redbus.in/bus-tickets/patna-to-bettiah',
 'https://www.redbus.in/bus-tickets/gopalganj-to-delhi',
 'https://www.redbus.in/bus-tickets/patna-to-motihari',
 'https://www.redbus.in/bus-tickets/delhi-to-motihari',
 'https://www.redbus.in/bus-tickets/bettiah-to-patna',
 'https://www.redbus.in/bus-tickets/motihari-to-delhi',
 'https://www.redbus.in/bus-tickets/patna-to-balmiki-nagar-bihar',
 'https://www.redbus.in/bus-tickets/balmiki-nagar-bihar-to-patna',
 'https://www.redbus.in/bus-tickets/patna-to-kathmandu',
 'https://www.redbus.in/bus-tickets/patna-to-katihar']

In [62]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

def scrape_bus_details(driver, route_links):
    details_list = []

    for link in route_links:
        driver.get(link)
        time.sleep(2)  # Allow the page to load initially
        
        # Get the body element for scrolling
        body = driver.find_element(By.TAG_NAME, "body")

        # Optional: Click on "View Buses" if needed
        try:
            view_buses = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, "//div[@class='button']"))
            )
            driver.execute_script("arguments[0].click();", view_buses)
        except Exception:
            # Suppress error if the 'View Buses' button is not found
            pass

        # Scrolling logic
        old_page_source = driver.page_source
        while True:
            body.send_keys(Keys.PAGE_DOWN)
            time.sleep(2)  # Wait for new data to load
            
            new_page_source = driver.page_source
            if new_page_source == old_page_source:
                break
            old_page_source = new_page_source

        # Collect bus details
        wait = WebDriverWait(driver, 30)
        try:
            bus_containers = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "bus-item-details")))
            for container in bus_containers:
                try:
                    bus_name = container.find_element(By.XPATH, ".//div[@class='travels lh-24 f-bold d-color']").text
                    bus_type = container.find_element(By.XPATH, ".//div[@class='bus-type f-12 m-top-16 l-color evBus']").text
                    departing_time = container.find_element(By.XPATH, ".//div[@class='dp-time f-19 d-color f-bold']").text
                    duration = container.find_element(By.XPATH, ".//div[@class='dur l-color lh-24']").text
                    reaching_time = container.find_element(By.XPATH, ".//div[@class='bp-time f-19 d-color disp-Inline']").text

                    try:
                        rating = container.find_element(By.CSS_SELECTOR, ".rating-sec .rating span").text
                    except Exception:
                        rating = "N/A"  # Default value if rating is not found

                    price = container.find_element(By.XPATH, ".//div[contains(@class, 'fare d-block')]").text
                    seats_available = container.find_element(By.CLASS_NAME, "seat-left").text

                    details_list.append({
                        "bus_name": bus_name,
                        "bus_type": bus_type,
                        "departing_time": departing_time,
                        "duration": duration,
                        "reaching_time": reaching_time,
                        "rating": rating,
                        "price": price,
                        "seats_available": seats_available
                    })

                except Exception:
                    # Suppress errors in extracting details from a container
                    pass
        except Exception:
            # Suppress errors in finding bus containers
            pass

    return details_list

# Initialize the WebDriver
driver = webdriver.Chrome()

# Example list of route links
route_links = ['https://www.redbus.in/bus-tickets/patna-to-bettiah',
 'https://www.redbus.in/bus-tickets/gopalganj-to-delhi',
 'https://www.redbus.in/bus-tickets/patna-to-motihari',
 'https://www.redbus.in/bus-tickets/delhi-to-motihari',
 'https://www.redbus.in/bus-tickets/bettiah-to-patna',
 'https://www.redbus.in/bus-tickets/motihari-to-delhi',
 'https://www.redbus.in/bus-tickets/patna-to-balmiki-nagar-bihar',
 'https://www.redbus.in/bus-tickets/balmiki-nagar-bihar-to-patna',
 'https://www.redbus.in/bus-tickets/patna-to-kathmandu',
 'https://www.redbus.in/bus-tickets/patna-to-katihar']
    


# Perform scraping
details_list = scrape_bus_details(driver, route_links)

# Print the collected details
print(details_list)

# Close the browser
driver.quit()


[{'bus_name': 'Jai Mata Di Tour and Travels', 'bus_type': 'A/C Seater / Sleeper (2+2)', 'departing_time': '20:28', 'duration': '05h 32m', 'reaching_time': '02:00', 'rating': '3.4', 'price': '450', 'seats_available': '6 Seats available'}, {'bus_name': 'Jai Hanuman Travels', 'bus_type': 'A/C Seater / Sleeper (2+2)', 'departing_time': '22:20', 'duration': '06h 40m', 'reaching_time': '05:00', 'rating': '3.5', 'price': 'INR 550', 'seats_available': '54 Seats available'}, {'bus_name': 'Jai Mata Di Tour and Travels', 'bus_type': 'A/C Seater / Sleeper (2+2)', 'departing_time': '22:40', 'duration': '05h 20m', 'reaching_time': '04:00', 'rating': '3.2', 'price': '405', 'seats_available': '28 Seats available'}, {'bus_name': 'V.I.P Travels', 'bus_type': 'A/C Seater / Sleeper (2+2)', 'departing_time': '21:40', 'duration': '06h 00m', 'reaching_time': '03:40', 'rating': '2.7', 'price': 'INR 350', 'seats_available': '61 Seats available'}, {'bus_name': 'ROYAL TRAVELS AND CARGO', 'bus_type': 'A/C Seater 

In [63]:
result5=pd.DataFrame(details_list)
result5

,bus_name,bus_type,departing_time,duration,reaching_time,rating,price,seats_available
0,Jai Mata Di Tour and Travels,A/C Seater / Sleeper (2+2),20:28,05h 32m,02:00,3.4,450,6 Seats available
1,Jai Hanuman Travels,A/C Seater / Sleeper (2+2),22:20,06h 40m,05:00,3.5,INR 550,54 Seats available
2,Jai Mata Di Tour and Travels,A/C Seater / Sleeper (2+2),22:40,05h 20m,04:00,3.2,405,28 Seats available
3,V.I.P Travels,A/C Seater / Sleeper (2+2),21:40,06h 00m,03:40,2.7,INR 350,61 Seats available
4,ROYAL TRAVELS AND CARGO,A/C Seater / Sleeper (2+2),17:30,15h 00m,08:30,2.2,INR 1896,9 Seats available
5,Shree Balajee Travels And Cargo,A/C Seater / Sleeper (2+2),20:00,18h 30m,14:30,1.3,INR 1899,60 Seats available
6,New Travel India,A/C Seater / Sleeper (2+1),18:15,13h 45m,08:00,1.2,2249,14 Seats available
7,New Travel India,A/C Seater / Sleeper (2+1),18:15,13h 55m,08:10,1.2,2249,30 Seats available
8,New Travel India,A/C Seater / Sleeper (2+1),17:35,14h 55m,08:30,1.2,2249,14 Seats available
9,New Travel India,A/C Seater / Sleeper (2+1),17:40,14h 50m,08:30,1.2,2249,14 Seats available


In [64]:
result5=pd.concat([route_details,result5],axis=1)
result5

,route_link,route_title,bus_name,bus_type,departing_time,duration,reaching_time,rating,price,seats_available
0,https://www.redbus.in/bus-tickets/patna-to-bet...,Patna (Bihar) to Bettiah,Jai Mata Di Tour and Travels,A/C Seater / Sleeper (2+2),20:28,05h 32m,02:00,3.4,450,6 Seats available
1,https://www.redbus.in/bus-tickets/gopalganj-to...,Gopalganj (Bihar) to Delhi,Jai Hanuman Travels,A/C Seater / Sleeper (2+2),22:20,06h 40m,05:00,3.5,INR 550,54 Seats available
2,https://www.redbus.in/bus-tickets/patna-to-mot...,Patna (Bihar) to Motihari,Jai Mata Di Tour and Travels,A/C Seater / Sleeper (2+2),22:40,05h 20m,04:00,3.2,405,28 Seats available
3,https://www.redbus.in/bus-tickets/delhi-to-mot...,Delhi to Motihari,V.I.P Travels,A/C Seater / Sleeper (2+2),21:40,06h 00m,03:40,2.7,INR 350,61 Seats available
4,https://www.redbus.in/bus-tickets/bettiah-to-p...,Bettiah to Patna (Bihar),ROYAL TRAVELS AND CARGO,A/C Seater / Sleeper (2+2),17:30,15h 00m,08:30,2.2,INR 1896,9 Seats available
5,https://www.redbus.in/bus-tickets/motihari-to-...,Motihari to Delhi,Shree Balajee Travels And Cargo,A/C Seater / Sleeper (2+2),20:00,18h 30m,14:30,1.3,INR 1899,60 Seats available
6,https://www.redbus.in/bus-tickets/patna-to-bal...,Patna (Bihar) to Balmiki Nagar (bihar),New Travel India,A/C Seater / Sleeper (2+1),18:15,13h 45m,08:00,1.2,2249,14 Seats available
7,https://www.redbus.in/bus-tickets/balmiki-naga...,Balmiki Nagar (bihar) to Patna (Bihar),New Travel India,A/C Seater / Sleeper (2+1),18:15,13h 55m,08:10,1.2,2249,30 Seats available
8,https://www.redbus.in/bus-tickets/patna-to-kat...,Patna (Bihar) to Kathmandu,New Travel India,A/C Seater / Sleeper (2+1),17:35,14h 55m,08:30,1.2,2249,14 Seats available
9,https://www.redbus.in/bus-tickets/patna-to-kat...,Patna (Bihar) to Katihar,New Travel India,A/C Seater / Sleeper (2+1),17:40,14h 50m,08:30,1.2,2249,14 Seats available


In [65]:
result5=result5.ffill()
result5

,route_link,route_title,bus_name,bus_type,departing_time,duration,reaching_time,rating,price,seats_available
0,https://www.redbus.in/bus-tickets/patna-to-bet...,Patna (Bihar) to Bettiah,Jai Mata Di Tour and Travels,A/C Seater / Sleeper (2+2),20:28,05h 32m,02:00,3.4,450,6 Seats available
1,https://www.redbus.in/bus-tickets/gopalganj-to...,Gopalganj (Bihar) to Delhi,Jai Hanuman Travels,A/C Seater / Sleeper (2+2),22:20,06h 40m,05:00,3.5,INR 550,54 Seats available
2,https://www.redbus.in/bus-tickets/patna-to-mot...,Patna (Bihar) to Motihari,Jai Mata Di Tour and Travels,A/C Seater / Sleeper (2+2),22:40,05h 20m,04:00,3.2,405,28 Seats available
3,https://www.redbus.in/bus-tickets/delhi-to-mot...,Delhi to Motihari,V.I.P Travels,A/C Seater / Sleeper (2+2),21:40,06h 00m,03:40,2.7,INR 350,61 Seats available
4,https://www.redbus.in/bus-tickets/bettiah-to-p...,Bettiah to Patna (Bihar),ROYAL TRAVELS AND CARGO,A/C Seater / Sleeper (2+2),17:30,15h 00m,08:30,2.2,INR 1896,9 Seats available
5,https://www.redbus.in/bus-tickets/motihari-to-...,Motihari to Delhi,Shree Balajee Travels And Cargo,A/C Seater / Sleeper (2+2),20:00,18h 30m,14:30,1.3,INR 1899,60 Seats available
6,https://www.redbus.in/bus-tickets/patna-to-bal...,Patna (Bihar) to Balmiki Nagar (bihar),New Travel India,A/C Seater / Sleeper (2+1),18:15,13h 45m,08:00,1.2,2249,14 Seats available
7,https://www.redbus.in/bus-tickets/balmiki-naga...,Balmiki Nagar (bihar) to Patna (Bihar),New Travel India,A/C Seater / Sleeper (2+1),18:15,13h 55m,08:10,1.2,2249,30 Seats available
8,https://www.redbus.in/bus-tickets/patna-to-kat...,Patna (Bihar) to Kathmandu,New Travel India,A/C Seater / Sleeper (2+1),17:35,14h 55m,08:30,1.2,2249,14 Seats available
9,https://www.redbus.in/bus-tickets/patna-to-kat...,Patna (Bihar) to Katihar,New Travel India,A/C Seater / Sleeper (2+1),17:40,14h 50m,08:30,1.2,2249,14 Seats available


In [66]:
ap_ts_ks_sb_wb_br = pd.concat([ap_ts_ks_sb_wb,result5], axis=0, ignore_index=True)

# Display the combined DataFrame
print(ap_ts_ks_sb_wb_br)

                                            route_link  \
0    https://www.redbus.in/bus-tickets/vijayawada-t...   
1    https://www.redbus.in/bus-tickets/hyderabad-to...   
2    https://www.redbus.in/bus-tickets/kakinada-to-...   
3    https://www.redbus.in/bus-tickets/visakhapatna...   
4    https://www.redbus.in/bus-tickets/chittoor-and...   
..                                                 ...   
566  https://www.redbus.in/bus-tickets/patna-to-kat...   
567  https://www.redbus.in/bus-tickets/patna-to-kat...   
568  https://www.redbus.in/bus-tickets/patna-to-kat...   
569  https://www.redbus.in/bus-tickets/patna-to-kat...   
570  https://www.redbus.in/bus-tickets/patna-to-kat...   

                                route_title  \
0                   Vijayawada to Hyderabad   
1                   Hyderabad to Vijayawada   
2                 Kakinada to Visakhapatnam   
3                 Visakhapatnam to Kakinada   
4    Chittoor (Andhra Pradesh) to Bangalore   
..                   

In [67]:
ap_ts_ks_sb_wb_br=pd.DataFrame(ap_ts_ks_sb_wb_br)
ap_ts_ks_sb_wb_br

,route_link,route_title,bus_name,bus_type,departing_time,duration,reaching_time,rating,price,seats_available
0,https://www.redbus.in/bus-tickets/vijayawada-t...,Vijayawada to Hyderabad,APSRTC - 35133,AMARAVATHI (VOLVO / SCANIA A.C Multi Axle),17:00,05h 50m,22:50,3.3,INR 720,33 Seats available
1,https://www.redbus.in/bus-tickets/hyderabad-to...,Hyderabad to Vijayawada,APSRTC - 35085,DOLPHIN CRUISE (VOLVO / SCANIA A.C Multi Axle),18:30,06h 00m,00:30,3.9,INR 720,38 Seats available
2,https://www.redbus.in/bus-tickets/kakinada-to-...,Kakinada to Visakhapatnam,APSRTC - 3613,Super Luxury (Non AC Seater 2+2 Push Back),19:00,07h 42m,02:42,3.1,INR 469,33 Seats available
3,https://www.redbus.in/bus-tickets/visakhapatna...,Visakhapatnam to Kakinada,APSRTC - 9316,Super Luxury (Non AC Seater 2+2 Push Back),20:20,07h 50m,04:10,4.1,INR 469,16 Seats available
4,https://www.redbus.in/bus-tickets/chittoor-and...,Chittoor (Andhra Pradesh) to Bangalore,APSRTC - 35189,DOLPHIN CRUISE (VOLVO / SCANIA A.C Multi Axle),20:30,04h 35m,01:05,3.8,INR 670,27 Seats available
...,...,...,...,...,...,...,...,...,...,...
566,https://www.redbus.in/bus-tickets/patna-to-kat...,Patna (Bihar) to Katihar,Jai Mata Di Tour and Travels,A/C Seater / Sleeper (2+2),18:30,10h 30m,05:00,2.3,450,10 Seats available
567,https://www.redbus.in/bus-tickets/patna-to-kat...,Patna (Bihar) to Katihar,Bihar state road transport corporation (BSRTC)...,AC Seater (2+2),21:25,09h 05m,06:30,3.3,INR 468,33 Seats available
568,https://www.redbus.in/bus-tickets/patna-to-kat...,Patna (Bihar) to Katihar,SOBHA TRAVELS (A.T),A/C Seater / Sleeper (2+1),20:40,08h 20m,05:00,2.5,540,28 Seats available
569,https://www.redbus.in/bus-tickets/patna-to-kat...,Patna (Bihar) to Katihar,Sri Krishna Rath,A/C Seater / Sleeper (2+2),20:40,09h 05m,05:45,1.8,675,26 Seats available


In [68]:
#ksrtc
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
route_links=[]
data = []

# Initialize WebDriver
driver = webdriver.Chrome()

try:
    driver.get("https://www.redbus.in/online-booking/hrtc/?utm_source=rtchometile")
    driver.maximize_window()
    
    # Initialize WebDriverWait
    wait = WebDriverWait(driver, 10)  # 10 seconds timeout
    
    # Wait for elements with the class name "route" to be present
    route_elements = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "route")))
    
    for route in route_elements:
        try:
            route_href = route.get_attribute('href')
            route_title = route.get_attribute('title')
            route_links.append(route_href)
            data.append({"route_link": route_href, "route_title": route_title})
        except TimeoutException as e:
            print("TimeoutException caught:", e)


finally:
    driver.quit()
# Print the collected data
for item in data:
    print(item)



{'route_link': 'https://www.redbus.in/bus-tickets/delhi-to-shimla', 'route_title': 'Delhi to Shimla'}
{'route_link': 'https://www.redbus.in/bus-tickets/shimla-to-delhi', 'route_title': 'Shimla to Delhi'}
{'route_link': 'https://www.redbus.in/bus-tickets/manali-to-chandigarh', 'route_title': 'Manali to Chandigarh'}
{'route_link': 'https://www.redbus.in/bus-tickets/chandigarh-to-manali', 'route_title': 'Chandigarh to Manali'}
{'route_link': 'https://www.redbus.in/bus-tickets/delhi-to-manali', 'route_title': 'Delhi to Manali'}
{'route_link': 'https://www.redbus.in/bus-tickets/hamirpur-himachal-pradesh-to-chandigarh', 'route_title': 'Hamirpur (Himachal Pradesh) to Chandigarh'}
{'route_link': 'https://www.redbus.in/bus-tickets/delhi-to-hamirpur-himachal-pradesh', 'route_title': 'Delhi to Hamirpur (Himachal Pradesh)'}
{'route_link': 'https://www.redbus.in/bus-tickets/delhi-to-chandigarh', 'route_title': 'Delhi to Chandigarh'}
{'route_link': 'https://www.redbus.in/bus-tickets/manali-to-delhi'

In [69]:
route_details=pd.DataFrame(data)
route_details

,route_link,route_title
0,https://www.redbus.in/bus-tickets/delhi-to-shimla,Delhi to Shimla
1,https://www.redbus.in/bus-tickets/shimla-to-delhi,Shimla to Delhi
2,https://www.redbus.in/bus-tickets/manali-to-ch...,Manali to Chandigarh
3,https://www.redbus.in/bus-tickets/chandigarh-t...,Chandigarh to Manali
4,https://www.redbus.in/bus-tickets/delhi-to-manali,Delhi to Manali
5,https://www.redbus.in/bus-tickets/hamirpur-him...,Hamirpur (Himachal Pradesh) to Chandigarh
6,https://www.redbus.in/bus-tickets/delhi-to-ham...,Delhi to Hamirpur (Himachal Pradesh)
7,https://www.redbus.in/bus-tickets/delhi-to-cha...,Delhi to Chandigarh
8,https://www.redbus.in/bus-tickets/manali-to-delhi,Manali to Delhi
9,https://www.redbus.in/bus-tickets/hamirpur-him...,Hamirpur (Himachal Pradesh) to Delhi


In [98]:
route_links

['https://www.redbus.in/bus-tickets/delhi-to-shimla',
 'https://www.redbus.in/bus-tickets/shimla-to-delhi',
 'https://www.redbus.in/bus-tickets/manali-to-chandigarh',
 'https://www.redbus.in/bus-tickets/chandigarh-to-manali',
 'https://www.redbus.in/bus-tickets/delhi-to-manali',
 'https://www.redbus.in/bus-tickets/hamirpur-himachal-pradesh-to-chandigarh',
 'https://www.redbus.in/bus-tickets/delhi-to-hamirpur-himachal-pradesh',
 'https://www.redbus.in/bus-tickets/delhi-to-chandigarh',
 'https://www.redbus.in/bus-tickets/manali-to-delhi',
 'https://www.redbus.in/bus-tickets/hamirpur-himachal-pradesh-to-delhi']

In [70]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

def scrape_bus_details(driver, route_links):
    details_list = []

    for link in route_links:
        driver.get(link)
        time.sleep(2)  # Allow the page to load initially
        
        # Get the body element for scrolling
        body = driver.find_element(By.TAG_NAME, "body")

        # Optional: Click on "View Buses" if needed
        try:
            view_buses = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, "//div[@class='button']"))
            )
            driver.execute_script("arguments[0].click();", view_buses)
        except Exception:
            # Suppress error if the 'View Buses' button is not found
            pass

        # Scrolling logic
        old_page_source = driver.page_source
        while True:
            body.send_keys(Keys.PAGE_DOWN)
            time.sleep(2)  # Wait for new data to load
            
            new_page_source = driver.page_source
            if new_page_source == old_page_source:
                break
            old_page_source = new_page_source

        # Collect bus details
        wait = WebDriverWait(driver, 30)
        try:
            bus_containers = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "bus-item-details")))
            for container in bus_containers:
                try:
                    bus_name = container.find_element(By.XPATH, ".//div[@class='travels lh-24 f-bold d-color']").text
                    bus_type = container.find_element(By.XPATH, ".//div[@class='bus-type f-12 m-top-16 l-color evBus']").text
                    departing_time = container.find_element(By.XPATH, ".//div[@class='dp-time f-19 d-color f-bold']").text
                    duration = container.find_element(By.XPATH, ".//div[@class='dur l-color lh-24']").text
                    reaching_time = container.find_element(By.XPATH, ".//div[@class='bp-time f-19 d-color disp-Inline']").text

                    try:
                        rating = container.find_element(By.CSS_SELECTOR, ".rating-sec .rating span").text
                    except Exception:
                        rating = "N/A"  # Default value if rating is not found

                    price = container.find_element(By.XPATH, ".//div[contains(@class, 'fare d-block')]").text
                    seats_available = container.find_element(By.CLASS_NAME, "seat-left").text

                    details_list.append({
                        "bus_name": bus_name,
                        "bus_type": bus_type,
                        "departing_time": departing_time,
                        "duration": duration,
                        "reaching_time": reaching_time,
                        "rating": rating,
                        "price": price,
                        "seats_available": seats_available
                    })

                except Exception:
                    # Suppress errors in extracting details from a container
                    pass
        except Exception:
            # Suppress errors in finding bus containers
            pass

    return details_list

# Initialize the WebDriver
driver = webdriver.Chrome()

# Example list of route links
route_links = ['https://www.redbus.in/bus-tickets/delhi-to-shimla',
 'https://www.redbus.in/bus-tickets/shimla-to-delhi',
 'https://www.redbus.in/bus-tickets/manali-to-chandigarh',
 'https://www.redbus.in/bus-tickets/chandigarh-to-manali',
 'https://www.redbus.in/bus-tickets/delhi-to-manali',
 'https://www.redbus.in/bus-tickets/hamirpur-himachal-pradesh-to-chandigarh',
 'https://www.redbus.in/bus-tickets/delhi-to-hamirpur-himachal-pradesh',
 'https://www.redbus.in/bus-tickets/delhi-to-chandigarh',
 'https://www.redbus.in/bus-tickets/manali-to-delhi',
 'https://www.redbus.in/bus-tickets/hamirpur-himachal-pradesh-to-delhi']
    


# Perform scraping
details_list = scrape_bus_details(driver, route_links)

# Print the collected details
print(details_list)

# Close the browser
driver.quit()


[{'bus_name': 'HRTC - 538', 'bus_type': 'Ordinary 3+2 Non AC Seater', 'departing_time': '18:31', 'duration': '10h 59m', 'reaching_time': '05:30', 'rating': '4.0', 'price': 'INR 507', 'seats_available': '27 Seats available'}, {'bus_name': 'HRTC - 93', 'bus_type': 'Himsuta AC Seater Volvo/Scania 2+2', 'departing_time': '19:35', 'duration': '09h 10m', 'reaching_time': '04:45', 'rating': '3.8', 'price': 'INR 892', 'seats_available': '14 Seats available'}, {'bus_name': 'HRTC - 301', 'bus_type': 'Ordinary 3+2 Non AC Seater', 'departing_time': '19:43', 'duration': '09h 47m', 'reaching_time': '05:30', 'rating': '4.2', 'price': 'INR 507', 'seats_available': '26 Seats available'}, {'bus_name': 'HRTC - 556', 'bus_type': 'Ordinary 3+2 Non AC Seater', 'departing_time': '20:10', 'duration': '09h 35m', 'reaching_time': '05:45', 'rating': '4.5', 'price': 'INR 507', 'seats_available': '27 Seats available'}, {'bus_name': 'HRTC - 591', 'bus_type': 'A/C Executive (2+3)', 'departing_time': '21:00', 'durati

In [71]:
result6=pd.DataFrame(details_list)
result6

,bus_name,bus_type,departing_time,duration,reaching_time,rating,price,seats_available
0,HRTC - 538,Ordinary 3+2 Non AC Seater,18:31,10h 59m,05:30,4.0,INR 507,27 Seats available
1,HRTC - 93,Himsuta AC Seater Volvo/Scania 2+2,19:35,09h 10m,04:45,3.8,INR 892,14 Seats available
2,HRTC - 301,Ordinary 3+2 Non AC Seater,19:43,09h 47m,05:30,4.2,INR 507,26 Seats available
3,HRTC - 556,Ordinary 3+2 Non AC Seater,20:10,09h 35m,05:45,4.5,INR 507,27 Seats available
4,HRTC - 591,A/C Executive (2+3),21:00,11h 15m,08:15,4.1,INR 622,34 Seats available
...,...,...,...,...,...,...,...,...
119,HRTC - 15,Himsuta AC Seater Volvo/Scania 2+2,20:00,08h 30m,04:30,3.5,INR 1008,11 Seats available
120,HRTC - 99,Himsuta AC Seater Volvo/Scania 2+2,21:00,10h 00m,07:00,4.1,INR 1118,13 Seats available
121,HRTC - 79,Himsuta AC Seater Volvo/Scania 2+2,21:30,09h 30m,07:00,3.7,INR 1115,18 Seats available
122,Ram Dalal Holidays,Volvo A/C Semi Sleeper (2+2),21:40,08h 20m,06:00,4.7,648,11 Seats available


In [72]:
result6=pd.concat([route_details,result6],axis=1)
result6

,route_link,route_title,bus_name,bus_type,departing_time,duration,reaching_time,rating,price,seats_available
0,https://www.redbus.in/bus-tickets/delhi-to-shimla,Delhi to Shimla,HRTC - 538,Ordinary 3+2 Non AC Seater,18:31,10h 59m,05:30,4.0,INR 507,27 Seats available
1,https://www.redbus.in/bus-tickets/shimla-to-delhi,Shimla to Delhi,HRTC - 93,Himsuta AC Seater Volvo/Scania 2+2,19:35,09h 10m,04:45,3.8,INR 892,14 Seats available
2,https://www.redbus.in/bus-tickets/manali-to-ch...,Manali to Chandigarh,HRTC - 301,Ordinary 3+2 Non AC Seater,19:43,09h 47m,05:30,4.2,INR 507,26 Seats available
3,https://www.redbus.in/bus-tickets/chandigarh-t...,Chandigarh to Manali,HRTC - 556,Ordinary 3+2 Non AC Seater,20:10,09h 35m,05:45,4.5,INR 507,27 Seats available
4,https://www.redbus.in/bus-tickets/delhi-to-manali,Delhi to Manali,HRTC - 591,A/C Executive (2+3),21:00,11h 15m,08:15,4.1,INR 622,34 Seats available
...,...,...,...,...,...,...,...,...,...,...
119,NaN,NaN,HRTC - 15,Himsuta AC Seater Volvo/Scania 2+2,20:00,08h 30m,04:30,3.5,INR 1008,11 Seats available
120,NaN,NaN,HRTC - 99,Himsuta AC Seater Volvo/Scania 2+2,21:00,10h 00m,07:00,4.1,INR 1118,13 Seats available
121,NaN,NaN,HRTC - 79,Himsuta AC Seater Volvo/Scania 2+2,21:30,09h 30m,07:00,3.7,INR 1115,18 Seats available
122,NaN,NaN,Ram Dalal Holidays,Volvo A/C Semi Sleeper (2+2),21:40,08h 20m,06:00,4.7,648,11 Seats available


In [73]:
result6=result6.ffill()
result6

,route_link,route_title,bus_name,bus_type,departing_time,duration,reaching_time,rating,price,seats_available
0,https://www.redbus.in/bus-tickets/delhi-to-shimla,Delhi to Shimla,HRTC - 538,Ordinary 3+2 Non AC Seater,18:31,10h 59m,05:30,4.0,INR 507,27 Seats available
1,https://www.redbus.in/bus-tickets/shimla-to-delhi,Shimla to Delhi,HRTC - 93,Himsuta AC Seater Volvo/Scania 2+2,19:35,09h 10m,04:45,3.8,INR 892,14 Seats available
2,https://www.redbus.in/bus-tickets/manali-to-ch...,Manali to Chandigarh,HRTC - 301,Ordinary 3+2 Non AC Seater,19:43,09h 47m,05:30,4.2,INR 507,26 Seats available
3,https://www.redbus.in/bus-tickets/chandigarh-t...,Chandigarh to Manali,HRTC - 556,Ordinary 3+2 Non AC Seater,20:10,09h 35m,05:45,4.5,INR 507,27 Seats available
4,https://www.redbus.in/bus-tickets/delhi-to-manali,Delhi to Manali,HRTC - 591,A/C Executive (2+3),21:00,11h 15m,08:15,4.1,INR 622,34 Seats available
...,...,...,...,...,...,...,...,...,...,...
119,https://www.redbus.in/bus-tickets/hamirpur-him...,Hamirpur (Himachal Pradesh) to Delhi,HRTC - 15,Himsuta AC Seater Volvo/Scania 2+2,20:00,08h 30m,04:30,3.5,INR 1008,11 Seats available
120,https://www.redbus.in/bus-tickets/hamirpur-him...,Hamirpur (Himachal Pradesh) to Delhi,HRTC - 99,Himsuta AC Seater Volvo/Scania 2+2,21:00,10h 00m,07:00,4.1,INR 1118,13 Seats available
121,https://www.redbus.in/bus-tickets/hamirpur-him...,Hamirpur (Himachal Pradesh) to Delhi,HRTC - 79,Himsuta AC Seater Volvo/Scania 2+2,21:30,09h 30m,07:00,3.7,INR 1115,18 Seats available
122,https://www.redbus.in/bus-tickets/hamirpur-him...,Hamirpur (Himachal Pradesh) to Delhi,Ram Dalal Holidays,Volvo A/C Semi Sleeper (2+2),21:40,08h 20m,06:00,4.7,648,11 Seats available


In [74]:
ap_ts_ks_sb_wb_bs_hr= pd.concat([result, result1,result2,result3,result4,result5,result6], axis=0, ignore_index=True)

# Display the combined DataFrame
print(ap_ts_ks_sb_wb_bs_hr)

                                            route_link  \
0    https://www.redbus.in/bus-tickets/vijayawada-t...   
1    https://www.redbus.in/bus-tickets/hyderabad-to...   
2    https://www.redbus.in/bus-tickets/kakinada-to-...   
3    https://www.redbus.in/bus-tickets/visakhapatna...   
4    https://www.redbus.in/bus-tickets/chittoor-and...   
..                                                 ...   
690  https://www.redbus.in/bus-tickets/hamirpur-him...   
691  https://www.redbus.in/bus-tickets/hamirpur-him...   
692  https://www.redbus.in/bus-tickets/hamirpur-him...   
693  https://www.redbus.in/bus-tickets/hamirpur-him...   
694  https://www.redbus.in/bus-tickets/hamirpur-him...   

                                route_title            bus_name  \
0                   Vijayawada to Hyderabad      APSRTC - 35133   
1                   Hyderabad to Vijayawada      APSRTC - 35085   
2                 Kakinada to Visakhapatnam       APSRTC - 3613   
3                 Visakhapatnam to 

In [75]:
ap_ts_ks_sb_wb_bs_hr=pd.DataFrame(ap_ts_ks_sb_wb_bs_hr)
ap_ts_ks_sb_wb_bs_hr

,route_link,route_title,bus_name,bus_type,departing_time,duration,reaching_time,rating,price,seats_available
0,https://www.redbus.in/bus-tickets/vijayawada-t...,Vijayawada to Hyderabad,APSRTC - 35133,AMARAVATHI (VOLVO / SCANIA A.C Multi Axle),17:00,05h 50m,22:50,3.3,INR 720,33 Seats available
1,https://www.redbus.in/bus-tickets/hyderabad-to...,Hyderabad to Vijayawada,APSRTC - 35085,DOLPHIN CRUISE (VOLVO / SCANIA A.C Multi Axle),18:30,06h 00m,00:30,3.9,INR 720,38 Seats available
2,https://www.redbus.in/bus-tickets/kakinada-to-...,Kakinada to Visakhapatnam,APSRTC - 3613,Super Luxury (Non AC Seater 2+2 Push Back),19:00,07h 42m,02:42,3.1,INR 469,33 Seats available
3,https://www.redbus.in/bus-tickets/visakhapatna...,Visakhapatnam to Kakinada,APSRTC - 9316,Super Luxury (Non AC Seater 2+2 Push Back),20:20,07h 50m,04:10,4.1,INR 469,16 Seats available
4,https://www.redbus.in/bus-tickets/chittoor-and...,Chittoor (Andhra Pradesh) to Bangalore,APSRTC - 35189,DOLPHIN CRUISE (VOLVO / SCANIA A.C Multi Axle),20:30,04h 35m,01:05,3.8,INR 670,27 Seats available
...,...,...,...,...,...,...,...,...,...,...
690,https://www.redbus.in/bus-tickets/hamirpur-him...,Hamirpur (Himachal Pradesh) to Delhi,HRTC - 15,Himsuta AC Seater Volvo/Scania 2+2,20:00,08h 30m,04:30,3.5,INR 1008,11 Seats available
691,https://www.redbus.in/bus-tickets/hamirpur-him...,Hamirpur (Himachal Pradesh) to Delhi,HRTC - 99,Himsuta AC Seater Volvo/Scania 2+2,21:00,10h 00m,07:00,4.1,INR 1118,13 Seats available
692,https://www.redbus.in/bus-tickets/hamirpur-him...,Hamirpur (Himachal Pradesh) to Delhi,HRTC - 79,Himsuta AC Seater Volvo/Scania 2+2,21:30,09h 30m,07:00,3.7,INR 1115,18 Seats available
693,https://www.redbus.in/bus-tickets/hamirpur-him...,Hamirpur (Himachal Pradesh) to Delhi,Ram Dalal Holidays,Volvo A/C Semi Sleeper (2+2),21:40,08h 20m,06:00,4.7,648,11 Seats available


In [76]:
#ksrtc
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
route_links=[]
data = []

# Initialize WebDriver
driver = webdriver.Chrome()

try:
    driver.get("https://www.redbus.in/online-booking/rsrtc/?utm_source=rtchometile")
    driver.maximize_window()
    
    # Initialize WebDriverWait
    wait = WebDriverWait(driver, 10)  # 10 seconds timeout
    
    # Wait for elements with the class name "route" to be present
    route_elements = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "route")))
    
    for route in route_elements:
        try:
            route_href = route.get_attribute('href')
            route_title = route.get_attribute('title')
            route_links.append(route_href)
            data.append({"route_link": route_href, "route_title": route_title})
        except TimeoutException as e:
            print("TimeoutException caught:", e)


finally:
    driver.quit()
# Print the collected data
for item in data:
    print(item)



{'route_link': 'https://www.redbus.in/bus-tickets/jodhpur-to-ajmer', 'route_title': 'Jodhpur to Ajmer'}
{'route_link': 'https://www.redbus.in/bus-tickets/beawer-to-jaipur', 'route_title': 'Beawar (Rajasthan) to Jaipur (Rajasthan)'}
{'route_link': 'https://www.redbus.in/bus-tickets/udaipur-to-jodhpur', 'route_title': 'Udaipur to Jodhpur'}
{'route_link': 'https://www.redbus.in/bus-tickets/jaipur-to-jodhpur', 'route_title': 'Jaipur (Rajasthan) to Jodhpur'}
{'route_link': 'https://www.redbus.in/bus-tickets/sikar-to-jaipur', 'route_title': 'Sikar to Jaipur (Rajasthan)'}
{'route_link': 'https://www.redbus.in/bus-tickets/kishangarh-to-jaipur', 'route_title': 'Kishangarh to Jaipur (Rajasthan)'}
{'route_link': 'https://www.redbus.in/bus-tickets/aligarh-uttar-pradesh-to-jaipur', 'route_title': 'Aligarh (uttar pradesh) to Jaipur (Rajasthan)'}
{'route_link': 'https://www.redbus.in/bus-tickets/jodhpur-to-beawer', 'route_title': 'Jodhpur to Beawar (Rajasthan)'}
{'route_link': 'https://www.redbus.in/

In [77]:
route_details=pd.DataFrame(data)
route_details

,route_link,route_title
0,https://www.redbus.in/bus-tickets/jodhpur-to-a...,Jodhpur to Ajmer
1,https://www.redbus.in/bus-tickets/beawer-to-ja...,Beawar (Rajasthan) to Jaipur (Rajasthan)
2,https://www.redbus.in/bus-tickets/udaipur-to-j...,Udaipur to Jodhpur
3,https://www.redbus.in/bus-tickets/jaipur-to-jo...,Jaipur (Rajasthan) to Jodhpur
4,https://www.redbus.in/bus-tickets/sikar-to-jaipur,Sikar to Jaipur (Rajasthan)
5,https://www.redbus.in/bus-tickets/kishangarh-t...,Kishangarh to Jaipur (Rajasthan)
6,https://www.redbus.in/bus-tickets/aligarh-utta...,Aligarh (uttar pradesh) to Jaipur (Rajasthan)
7,https://www.redbus.in/bus-tickets/jodhpur-to-b...,Jodhpur to Beawar (Rajasthan)
8,https://www.redbus.in/bus-tickets/kota-rajasth...,Kota(Rajasthan) to Jaipur (Rajasthan)
9,https://www.redbus.in/bus-tickets/jaipur-to-al...,Jaipur (Rajasthan) to Aligarh (uttar pradesh)


In [78]:
route_links

['https://www.redbus.in/bus-tickets/jodhpur-to-ajmer',
 'https://www.redbus.in/bus-tickets/beawer-to-jaipur',
 'https://www.redbus.in/bus-tickets/udaipur-to-jodhpur',
 'https://www.redbus.in/bus-tickets/jaipur-to-jodhpur',
 'https://www.redbus.in/bus-tickets/sikar-to-jaipur',
 'https://www.redbus.in/bus-tickets/kishangarh-to-jaipur',
 'https://www.redbus.in/bus-tickets/aligarh-uttar-pradesh-to-jaipur',
 'https://www.redbus.in/bus-tickets/jodhpur-to-beawer',
 'https://www.redbus.in/bus-tickets/kota-rajasthan-to-jaipur',
 'https://www.redbus.in/bus-tickets/jaipur-to-aligarh-uttar-pradesh']

In [79]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

def scrape_bus_details(driver, route_links):
    details_list = []

    for link in route_links:
        driver.get(link)
        time.sleep(2)  # Allow the page to load initially
        
        # Get the body element for scrolling
        body = driver.find_element(By.TAG_NAME, "body")

        # Optional: Click on "View Buses" if needed
        try:
            view_buses = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, "//div[@class='button']"))
            )
            driver.execute_script("arguments[0].click();", view_buses)
        except Exception:
            # Suppress error if the 'View Buses' button is not found
            pass

        # Scrolling logic
        old_page_source = driver.page_source
        while True:
            body.send_keys(Keys.PAGE_DOWN)
            time.sleep(2)  # Wait for new data to load
            
            new_page_source = driver.page_source
            if new_page_source == old_page_source:
                break
            old_page_source = new_page_source

        # Collect bus details
        wait = WebDriverWait(driver, 30)
        try:
            bus_containers = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "bus-item-details")))
            for container in bus_containers:
                try:
                    bus_name = container.find_element(By.XPATH, ".//div[@class='travels lh-24 f-bold d-color']").text
                    bus_type = container.find_element(By.XPATH, ".//div[@class='bus-type f-12 m-top-16 l-color evBus']").text
                    departing_time = container.find_element(By.XPATH, ".//div[@class='dp-time f-19 d-color f-bold']").text
                    duration = container.find_element(By.XPATH, ".//div[@class='dur l-color lh-24']").text
                    reaching_time = container.find_element(By.XPATH, ".//div[@class='bp-time f-19 d-color disp-Inline']").text

                    try:
                        rating = container.find_element(By.CSS_SELECTOR, ".rating-sec .rating span").text
                    except Exception:
                        rating = "N/A"  # Default value if rating is not found

                    price = container.find_element(By.XPATH, ".//div[contains(@class, 'fare d-block')]").text
                    seats_available = container.find_element(By.CLASS_NAME, "seat-left").text

                    details_list.append({
                        "bus_name": bus_name,
                        "bus_type": bus_type,
                        "departing_time": departing_time,
                        "duration": duration,
                        "reaching_time": reaching_time,
                        "rating": rating,
                        "price": price,
                        "seats_available": seats_available
                    })

                except Exception:
                    # Suppress errors in extracting details from a container
                    pass
        except Exception:
            # Suppress errors in finding bus containers
            pass

    return details_list

# Initialize the WebDriver
driver = webdriver.Chrome()

# Example list of route links
route_links = ['https://www.redbus.in/bus-tickets/jodhpur-to-ajmer',
 'https://www.redbus.in/bus-tickets/beawer-to-jaipur',
 'https://www.redbus.in/bus-tickets/udaipur-to-jodhpur',
 'https://www.redbus.in/bus-tickets/jaipur-to-jodhpur',
 'https://www.redbus.in/bus-tickets/sikar-to-jaipur',
 'https://www.redbus.in/bus-tickets/kishangarh-to-jaipur',
 'https://www.redbus.in/bus-tickets/aligarh-uttar-pradesh-to-jaipur',
 'https://www.redbus.in/bus-tickets/jodhpur-to-beawer',
 'https://www.redbus.in/bus-tickets/kota-rajasthan-to-jaipur',
 'https://www.redbus.in/bus-tickets/jaipur-to-aligarh-uttar-pradesh']
    


# Perform scraping
details_list = scrape_bus_details(driver, route_links)

# Print the collected details
print(details_list)

# Close the browser
driver.quit()


[{'bus_name': 'RSRTC - 160785', 'bus_type': 'Super Luxury Volvo AC Seater Pushback 2+2', 'departing_time': '17:30', 'duration': '04h 00m', 'reaching_time': '21:30', 'rating': '4.1', 'price': 'INR 460', 'seats_available': '21 Seats available'}, {'bus_name': 'RSRTC - 190951', 'bus_type': 'Express Non AC Seater 2+3', 'departing_time': '18:00', 'duration': '04h 30m', 'reaching_time': '22:30', 'rating': '4.2', 'price': 'INR 229', 'seats_available': '42 Seats available'}, {'bus_name': 'RSRTC - 193481', 'bus_type': 'Express Non AC Seater 2+3', 'departing_time': '18:30', 'duration': '04h 30m', 'reaching_time': '23:00', 'rating': '4.2', 'price': 'INR 229', 'seats_available': '42 Seats available'}, {'bus_name': 'RSRTC - 187140', 'bus_type': 'NON AC Seater / Sleeper 2+1', 'departing_time': '19:30', 'duration': '04h 00m', 'reaching_time': '23:30', 'rating': '4.5', 'price': 'INR 302', 'seats_available': '37 Seats available'}, {'bus_name': 'RSRTC - 176394', 'bus_type': 'Super Luxury Volvo AC Seater 

In [80]:
result7=pd.DataFrame(details_list)
result7

,bus_name,bus_type,departing_time,duration,reaching_time,rating,price,seats_available
0,RSRTC - 160785,Super Luxury Volvo AC Seater Pushback 2+2,17:30,04h 00m,21:30,4.1,INR 460,21 Seats available
1,RSRTC - 190951,Express Non AC Seater 2+3,18:00,04h 30m,22:30,4.2,INR 229,42 Seats available
2,RSRTC - 193481,Express Non AC Seater 2+3,18:30,04h 30m,23:00,4.2,INR 229,42 Seats available
3,RSRTC - 187140,NON AC Seater / Sleeper 2+1,19:30,04h 00m,23:30,4.5,INR 302,37 Seats available
4,RSRTC - 176394,Super Luxury Volvo AC Seater Pushback 2+2,20:00,03h 45m,23:45,4.4,INR 460,23 Seats available
...,...,...,...,...,...,...,...,...
126,,,,,,,,
127,,,,,,,,
128,,,,,,,,
129,,,,,,,,


In [81]:
result7=pd.concat([route_details,result7],axis=1)
result7

,route_link,route_title,bus_name,bus_type,departing_time,duration,reaching_time,rating,price,seats_available
0,https://www.redbus.in/bus-tickets/jodhpur-to-a...,Jodhpur to Ajmer,RSRTC - 160785,Super Luxury Volvo AC Seater Pushback 2+2,17:30,04h 00m,21:30,4.1,INR 460,21 Seats available
1,https://www.redbus.in/bus-tickets/beawer-to-ja...,Beawar (Rajasthan) to Jaipur (Rajasthan),RSRTC - 190951,Express Non AC Seater 2+3,18:00,04h 30m,22:30,4.2,INR 229,42 Seats available
2,https://www.redbus.in/bus-tickets/udaipur-to-j...,Udaipur to Jodhpur,RSRTC - 193481,Express Non AC Seater 2+3,18:30,04h 30m,23:00,4.2,INR 229,42 Seats available
3,https://www.redbus.in/bus-tickets/jaipur-to-jo...,Jaipur (Rajasthan) to Jodhpur,RSRTC - 187140,NON AC Seater / Sleeper 2+1,19:30,04h 00m,23:30,4.5,INR 302,37 Seats available
4,https://www.redbus.in/bus-tickets/sikar-to-jaipur,Sikar to Jaipur (Rajasthan),RSRTC - 176394,Super Luxury Volvo AC Seater Pushback 2+2,20:00,03h 45m,23:45,4.4,INR 460,23 Seats available
...,...,...,...,...,...,...,...,...,...,...
126,NaN,NaN,,,,,,,,
127,NaN,NaN,,,,,,,,
128,NaN,NaN,,,,,,,,
129,NaN,NaN,,,,,,,,


In [82]:
result7=result7.ffill()
result7

,route_link,route_title,bus_name,bus_type,departing_time,duration,reaching_time,rating,price,seats_available
0,https://www.redbus.in/bus-tickets/jodhpur-to-a...,Jodhpur to Ajmer,RSRTC - 160785,Super Luxury Volvo AC Seater Pushback 2+2,17:30,04h 00m,21:30,4.1,INR 460,21 Seats available
1,https://www.redbus.in/bus-tickets/beawer-to-ja...,Beawar (Rajasthan) to Jaipur (Rajasthan),RSRTC - 190951,Express Non AC Seater 2+3,18:00,04h 30m,22:30,4.2,INR 229,42 Seats available
2,https://www.redbus.in/bus-tickets/udaipur-to-j...,Udaipur to Jodhpur,RSRTC - 193481,Express Non AC Seater 2+3,18:30,04h 30m,23:00,4.2,INR 229,42 Seats available
3,https://www.redbus.in/bus-tickets/jaipur-to-jo...,Jaipur (Rajasthan) to Jodhpur,RSRTC - 187140,NON AC Seater / Sleeper 2+1,19:30,04h 00m,23:30,4.5,INR 302,37 Seats available
4,https://www.redbus.in/bus-tickets/sikar-to-jaipur,Sikar to Jaipur (Rajasthan),RSRTC - 176394,Super Luxury Volvo AC Seater Pushback 2+2,20:00,03h 45m,23:45,4.4,INR 460,23 Seats available
...,...,...,...,...,...,...,...,...,...,...
126,https://www.redbus.in/bus-tickets/jaipur-to-al...,Jaipur (Rajasthan) to Aligarh (uttar pradesh),,,,,,,,
127,https://www.redbus.in/bus-tickets/jaipur-to-al...,Jaipur (Rajasthan) to Aligarh (uttar pradesh),,,,,,,,
128,https://www.redbus.in/bus-tickets/jaipur-to-al...,Jaipur (Rajasthan) to Aligarh (uttar pradesh),,,,,,,,
129,https://www.redbus.in/bus-tickets/jaipur-to-al...,Jaipur (Rajasthan) to Aligarh (uttar pradesh),,,,,,,,


In [83]:
ap_ts_ks_sb_wb_bs_hr_rs= pd.concat([result, result1,result2,result3,result4,result5,result6,result7], axis=0, ignore_index=True)

# Display the combined DataFrame
print(ap_ts_ks_sb_wb_bs_hr_rs)

                                            route_link  \
0    https://www.redbus.in/bus-tickets/vijayawada-t...   
1    https://www.redbus.in/bus-tickets/hyderabad-to...   
2    https://www.redbus.in/bus-tickets/kakinada-to-...   
3    https://www.redbus.in/bus-tickets/visakhapatna...   
4    https://www.redbus.in/bus-tickets/chittoor-and...   
..                                                 ...   
821  https://www.redbus.in/bus-tickets/jaipur-to-al...   
822  https://www.redbus.in/bus-tickets/jaipur-to-al...   
823  https://www.redbus.in/bus-tickets/jaipur-to-al...   
824  https://www.redbus.in/bus-tickets/jaipur-to-al...   
825  https://www.redbus.in/bus-tickets/jaipur-to-al...   

                                       route_title        bus_name  \
0                          Vijayawada to Hyderabad  APSRTC - 35133   
1                          Hyderabad to Vijayawada  APSRTC - 35085   
2                        Kakinada to Visakhapatnam   APSRTC - 3613   
3                      

In [87]:
ap_ts_ks_sb_wb_bs_hr_rs=pd.DataFrame(ap_ts_ks_sb_wb_bs_hr_rs)
ap_ts_ks_sb_wb_bs_hr_rs

,route_link,route_title,bus_name,bus_type,departing_time,duration,reaching_time,rating,price,seats_available
0,https://www.redbus.in/bus-tickets/vijayawada-t...,Vijayawada to Hyderabad,APSRTC - 35133,AMARAVATHI (VOLVO / SCANIA A.C Multi Axle),17:00,05h 50m,22:50,3.3,INR 720,33 Seats available
1,https://www.redbus.in/bus-tickets/hyderabad-to...,Hyderabad to Vijayawada,APSRTC - 35085,DOLPHIN CRUISE (VOLVO / SCANIA A.C Multi Axle),18:30,06h 00m,00:30,3.9,INR 720,38 Seats available
2,https://www.redbus.in/bus-tickets/kakinada-to-...,Kakinada to Visakhapatnam,APSRTC - 3613,Super Luxury (Non AC Seater 2+2 Push Back),19:00,07h 42m,02:42,3.1,INR 469,33 Seats available
3,https://www.redbus.in/bus-tickets/visakhapatna...,Visakhapatnam to Kakinada,APSRTC - 9316,Super Luxury (Non AC Seater 2+2 Push Back),20:20,07h 50m,04:10,4.1,INR 469,16 Seats available
4,https://www.redbus.in/bus-tickets/chittoor-and...,Chittoor (Andhra Pradesh) to Bangalore,APSRTC - 35189,DOLPHIN CRUISE (VOLVO / SCANIA A.C Multi Axle),20:30,04h 35m,01:05,3.8,INR 670,27 Seats available
...,...,...,...,...,...,...,...,...,...,...
821,https://www.redbus.in/bus-tickets/jaipur-to-al...,Jaipur (Rajasthan) to Aligarh (uttar pradesh),,,,,,,,
822,https://www.redbus.in/bus-tickets/jaipur-to-al...,Jaipur (Rajasthan) to Aligarh (uttar pradesh),,,,,,,,
823,https://www.redbus.in/bus-tickets/jaipur-to-al...,Jaipur (Rajasthan) to Aligarh (uttar pradesh),,,,,,,,
824,https://www.redbus.in/bus-tickets/jaipur-to-al...,Jaipur (Rajasthan) to Aligarh (uttar pradesh),,,,,,,,


In [90]:
ap_ts_ks_sb_wb_bs_hr_rs=ap_ts_ks_sb_wb_bs_hr_rs.ffill()
ap_ts_ks_sb_wb_bs_hr_rs

,route_link,route_title,bus_name,bus_type,departing_time,duration,reaching_time,rating,price,seats_available
0,https://www.redbus.in/bus-tickets/vijayawada-t...,Vijayawada to Hyderabad,APSRTC - 35133,AMARAVATHI (VOLVO / SCANIA A.C Multi Axle),17:00,05h 50m,22:50,3.3,INR 720,33 Seats available
1,https://www.redbus.in/bus-tickets/hyderabad-to...,Hyderabad to Vijayawada,APSRTC - 35085,DOLPHIN CRUISE (VOLVO / SCANIA A.C Multi Axle),18:30,06h 00m,00:30,3.9,INR 720,38 Seats available
2,https://www.redbus.in/bus-tickets/kakinada-to-...,Kakinada to Visakhapatnam,APSRTC - 3613,Super Luxury (Non AC Seater 2+2 Push Back),19:00,07h 42m,02:42,3.1,INR 469,33 Seats available
3,https://www.redbus.in/bus-tickets/visakhapatna...,Visakhapatnam to Kakinada,APSRTC - 9316,Super Luxury (Non AC Seater 2+2 Push Back),20:20,07h 50m,04:10,4.1,INR 469,16 Seats available
4,https://www.redbus.in/bus-tickets/chittoor-and...,Chittoor (Andhra Pradesh) to Bangalore,APSRTC - 35189,DOLPHIN CRUISE (VOLVO / SCANIA A.C Multi Axle),20:30,04h 35m,01:05,3.8,INR 670,27 Seats available
...,...,...,...,...,...,...,...,...,...,...
821,https://www.redbus.in/bus-tickets/jaipur-to-al...,Jaipur (Rajasthan) to Aligarh (uttar pradesh),,,,,,,,
822,https://www.redbus.in/bus-tickets/jaipur-to-al...,Jaipur (Rajasthan) to Aligarh (uttar pradesh),,,,,,,,
823,https://www.redbus.in/bus-tickets/jaipur-to-al...,Jaipur (Rajasthan) to Aligarh (uttar pradesh),,,,,,,,
824,https://www.redbus.in/bus-tickets/jaipur-to-al...,Jaipur (Rajasthan) to Aligarh (uttar pradesh),,,,,,,,


In [91]:
#ksrtc
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
route_links=[]
data = []

# Initialize WebDriver
driver = webdriver.Chrome()

try:
    driver.get("https://www.redbus.in/online-booking/astc/?utm_source=rtchometile")
    driver.maximize_window()
    
    # Initialize WebDriverWait
    wait = WebDriverWait(driver, 10)  # 10 seconds timeout
    
    # Wait for elements with the class name "route" to be present
    route_elements = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "route")))
    
    for route in route_elements:
        try:
            route_href = route.get_attribute('href')
            route_title = route.get_attribute('title')
            route_links.append(route_href)
            data.append({"route_link": route_href, "route_title": route_title})
        except TimeoutException as e:
            print("TimeoutException caught:", e)


finally:
    driver.quit()
# Print the collected data
for item in data:
    print(item)



{'route_link': 'https://www.redbus.in/bus-tickets/tezpur-to-guwahati', 'route_title': 'Tezpur to Guwahati'}
{'route_link': 'https://www.redbus.in/bus-tickets/guwahati-to-tezpur', 'route_title': 'Guwahati to Tezpur'}
{'route_link': 'https://www.redbus.in/bus-tickets/nagaon-to-guwahati', 'route_title': 'Nagaon (Assam) to Guwahati'}
{'route_link': 'https://www.redbus.in/bus-tickets/guwahati-to-nagaon', 'route_title': 'Guwahati to Nagaon (Assam)'}
{'route_link': 'https://www.redbus.in/bus-tickets/goalpara-to-guwahati', 'route_title': 'Goalpara to Guwahati'}
{'route_link': 'https://www.redbus.in/bus-tickets/jorhat-to-north-lakhimpur', 'route_title': 'Jorhat to North Lakhimpur'}
{'route_link': 'https://www.redbus.in/bus-tickets/dhubri-to-guwahati', 'route_title': 'Dhubri to Guwahati'}
{'route_link': 'https://www.redbus.in/bus-tickets/guwahati-to-dhubri', 'route_title': 'Guwahati to Dhubri'}
{'route_link': 'https://www.redbus.in/bus-tickets/north-lakhimpur-to-sibsagar', 'route_title': 'North 

In [92]:
route_details=pd.DataFrame(data)
route_details

,route_link,route_title
0,https://www.redbus.in/bus-tickets/tezpur-to-gu...,Tezpur to Guwahati
1,https://www.redbus.in/bus-tickets/guwahati-to-...,Guwahati to Tezpur
2,https://www.redbus.in/bus-tickets/nagaon-to-gu...,Nagaon (Assam) to Guwahati
3,https://www.redbus.in/bus-tickets/guwahati-to-...,Guwahati to Nagaon (Assam)
4,https://www.redbus.in/bus-tickets/goalpara-to-...,Goalpara to Guwahati
5,https://www.redbus.in/bus-tickets/jorhat-to-no...,Jorhat to North Lakhimpur
6,https://www.redbus.in/bus-tickets/dhubri-to-gu...,Dhubri to Guwahati
7,https://www.redbus.in/bus-tickets/guwahati-to-...,Guwahati to Dhubri
8,https://www.redbus.in/bus-tickets/north-lakhim...,North Lakhimpur to Sibsagar
9,https://www.redbus.in/bus-tickets/north-lakhim...,North Lakhimpur to Jorhat


In [93]:
route_links

['https://www.redbus.in/bus-tickets/tezpur-to-guwahati',
 'https://www.redbus.in/bus-tickets/guwahati-to-tezpur',
 'https://www.redbus.in/bus-tickets/nagaon-to-guwahati',
 'https://www.redbus.in/bus-tickets/guwahati-to-nagaon',
 'https://www.redbus.in/bus-tickets/goalpara-to-guwahati',
 'https://www.redbus.in/bus-tickets/jorhat-to-north-lakhimpur',
 'https://www.redbus.in/bus-tickets/dhubri-to-guwahati',
 'https://www.redbus.in/bus-tickets/guwahati-to-dhubri',
 'https://www.redbus.in/bus-tickets/north-lakhimpur-to-sibsagar',
 'https://www.redbus.in/bus-tickets/north-lakhimpur-to-jorhat']

In [94]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

def scrape_bus_details(driver, route_links):
    details_list = []

    for link in route_links:
        driver.get(link)
        time.sleep(2)  # Allow the page to load initially
        
        # Get the body element for scrolling
        body = driver.find_element(By.TAG_NAME, "body")

        # Optional: Click on "View Buses" if needed
        try:
            view_buses = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, "//div[@class='button']"))
            )
            driver.execute_script("arguments[0].click();", view_buses)
        except Exception:
            # Suppress error if the 'View Buses' button is not found
            pass

        # Scrolling logic
        old_page_source = driver.page_source
        while True:
            body.send_keys(Keys.PAGE_DOWN)
            time.sleep(2)  # Wait for new data to load
            
            new_page_source = driver.page_source
            if new_page_source == old_page_source:
                break
            old_page_source = new_page_source

        # Collect bus details
        wait = WebDriverWait(driver, 30)
        try:
            bus_containers = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "bus-item-details")))
            for container in bus_containers:
                try:
                    bus_name = container.find_element(By.XPATH, ".//div[@class='travels lh-24 f-bold d-color']").text
                    bus_type = container.find_element(By.XPATH, ".//div[@class='bus-type f-12 m-top-16 l-color evBus']").text
                    departing_time = container.find_element(By.XPATH, ".//div[@class='dp-time f-19 d-color f-bold']").text
                    duration = container.find_element(By.XPATH, ".//div[@class='dur l-color lh-24']").text
                    reaching_time = container.find_element(By.XPATH, ".//div[@class='bp-time f-19 d-color disp-Inline']").text

                    try:
                        rating = container.find_element(By.CSS_SELECTOR, ".rating-sec .rating span").text
                    except Exception:
                        rating = "N/A"  # Default value if rating is not found

                    price = container.find_element(By.XPATH, ".//div[contains(@class, 'fare d-block')]").text
                    seats_available = container.find_element(By.CLASS_NAME, "seat-left").text

                    details_list.append({
                        "bus_name": bus_name,
                        "bus_type": bus_type,
                        "departing_time": departing_time,
                        "duration": duration,
                        "reaching_time": reaching_time,
                        "rating": rating,
                        "price": price,
                        "seats_available": seats_available
                    })

                except Exception:
                    # Suppress errors in extracting details from a container
                    pass
        except Exception:
            # Suppress errors in finding bus containers
            pass

    return details_list

# Initialize the WebDriver
driver = webdriver.Chrome()

# Example list of route links
route_links = ['https://www.redbus.in/bus-tickets/tezpur-to-guwahati',
 'https://www.redbus.in/bus-tickets/guwahati-to-tezpur',
 'https://www.redbus.in/bus-tickets/nagaon-to-guwahati',
 'https://www.redbus.in/bus-tickets/guwahati-to-nagaon',
 'https://www.redbus.in/bus-tickets/goalpara-to-guwahati',
 'https://www.redbus.in/bus-tickets/jorhat-to-north-lakhimpur',
 'https://www.redbus.in/bus-tickets/dhubri-to-guwahati',
 'https://www.redbus.in/bus-tickets/guwahati-to-dhubri',
 'https://www.redbus.in/bus-tickets/north-lakhimpur-to-sibsagar',
 'https://www.redbus.in/bus-tickets/north-lakhimpur-to-jorhat']


# Perform scraping
details_list = scrape_bus_details(driver, route_links)

# Print the collected details
print(details_list)

# Close the browser
driver.quit()


[{'bus_name': 'BLUE HILL TRAVELS INDIA LIMITED', 'bus_type': 'NON A/C Seater Semi Sleeper (2+1)', 'departing_time': '21:30', 'duration': '06h 00m', 'reaching_time': '03:30', 'rating': '3.2', 'price': 'INR 400', 'seats_available': '11 Seats available'}, {'bus_name': 'Himalayan Travels', 'bus_type': 'NON AC Seater / Sleeper 2+1', 'departing_time': '23:40', 'duration': '04h 20m', 'reaching_time': '04:00', 'rating': '2.2', 'price': '450', 'seats_available': '27 Seats available'}, {'bus_name': 'BLUE HILL TRAVELS INDIA LIMITED', 'bus_type': 'NON A/C Seater Semi Sleeper (2+1)', 'departing_time': '22:15', 'duration': '05h 30m', 'reaching_time': '03:45', 'rating': '3.7', 'price': 'INR 550', 'seats_available': '18 Seats available'}, {'bus_name': 'Yatra Travels', 'bus_type': 'NON A/C Seater (2+1)', 'departing_time': '18:00', 'duration': '05h 30m', 'reaching_time': '23:30', 'rating': '3.4', 'price': '400', 'seats_available': '13 Seats available'}, {'bus_name': 'Yatra Travels', 'bus_type': 'NON AC 

In [95]:
df=pd.DataFrame(details_list)
df

,bus_name,bus_type,departing_time,duration,reaching_time,rating,price,seats_available
0,BLUE HILL TRAVELS INDIA LIMITED,NON A/C Seater Semi Sleeper (2+1),21:30,06h 00m,03:30,3.2,INR 400,11 Seats available
1,Himalayan Travels,NON AC Seater / Sleeper 2+1,23:40,04h 20m,04:00,2.2,450,27 Seats available
2,BLUE HILL TRAVELS INDIA LIMITED,NON A/C Seater Semi Sleeper (2+1),22:15,05h 30m,03:45,3.7,INR 550,18 Seats available
3,Yatra Travels,NON A/C Seater (2+1),18:00,05h 30m,23:30,3.4,400,13 Seats available
4,Yatra Travels,NON AC Seater/ Sleeper (2+1),20:00,04h 00m,00:00,4.3,400,5 Seats available
5,Pranjit Travels (Under ASTC),A/C Seater / Sleeper (2+1),20:45,05h 45m,02:30,3.3,INR 500,16 Seats available
6,Pranjit Travels (Under ASTC),NON A/C Seater (2+1),21:45,04h 15m,02:00,3.9,INR 450,14 Seats available
7,Kalpana Travels,AC Seater (2+2),16:50,03h 25m,20:15,2.1,342,43 Seats available
8,Brindaban Travels,Non AC Seater (2+2),17:15,04h 00m,21:15,1.8,INR 375,35 Seats available
9,BLUE HILL TRAVELS INDIA LIMITED,NON A/C Seater Semi Sleeper (2+1),20:00,05h 15m,01:15,2.5,INR 500,21 Seats available


In [96]:
df=pd.concat([route_details,df])
df

,route_link,route_title,bus_name,bus_type,departing_time,duration,reaching_time,rating,price,seats_available
0,https://www.redbus.in/bus-tickets/tezpur-to-gu...,Tezpur to Guwahati,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://www.redbus.in/bus-tickets/guwahati-to-...,Guwahati to Tezpur,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://www.redbus.in/bus-tickets/nagaon-to-gu...,Nagaon (Assam) to Guwahati,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://www.redbus.in/bus-tickets/guwahati-to-...,Guwahati to Nagaon (Assam),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://www.redbus.in/bus-tickets/goalpara-to-...,Goalpara to Guwahati,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,https://www.redbus.in/bus-tickets/jorhat-to-no...,Jorhat to North Lakhimpur,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,https://www.redbus.in/bus-tickets/dhubri-to-gu...,Dhubri to Guwahati,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,https://www.redbus.in/bus-tickets/guwahati-to-...,Guwahati to Dhubri,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,https://www.redbus.in/bus-tickets/north-lakhim...,North Lakhimpur to Sibsagar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,https://www.redbus.in/bus-tickets/north-lakhim...,North Lakhimpur to Jorhat,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [97]:
result8 = df.bfill().ffill()
result8

,route_link,route_title,bus_name,bus_type,departing_time,duration,reaching_time,rating,price,seats_available
0,https://www.redbus.in/bus-tickets/tezpur-to-gu...,Tezpur to Guwahati,BLUE HILL TRAVELS INDIA LIMITED,NON A/C Seater Semi Sleeper (2+1),21:30,06h 00m,03:30,3.2,INR 400,11 Seats available
1,https://www.redbus.in/bus-tickets/guwahati-to-...,Guwahati to Tezpur,BLUE HILL TRAVELS INDIA LIMITED,NON A/C Seater Semi Sleeper (2+1),21:30,06h 00m,03:30,3.2,INR 400,11 Seats available
2,https://www.redbus.in/bus-tickets/nagaon-to-gu...,Nagaon (Assam) to Guwahati,BLUE HILL TRAVELS INDIA LIMITED,NON A/C Seater Semi Sleeper (2+1),21:30,06h 00m,03:30,3.2,INR 400,11 Seats available
3,https://www.redbus.in/bus-tickets/guwahati-to-...,Guwahati to Nagaon (Assam),BLUE HILL TRAVELS INDIA LIMITED,NON A/C Seater Semi Sleeper (2+1),21:30,06h 00m,03:30,3.2,INR 400,11 Seats available
4,https://www.redbus.in/bus-tickets/goalpara-to-...,Goalpara to Guwahati,BLUE HILL TRAVELS INDIA LIMITED,NON A/C Seater Semi Sleeper (2+1),21:30,06h 00m,03:30,3.2,INR 400,11 Seats available
5,https://www.redbus.in/bus-tickets/jorhat-to-no...,Jorhat to North Lakhimpur,BLUE HILL TRAVELS INDIA LIMITED,NON A/C Seater Semi Sleeper (2+1),21:30,06h 00m,03:30,3.2,INR 400,11 Seats available
6,https://www.redbus.in/bus-tickets/dhubri-to-gu...,Dhubri to Guwahati,BLUE HILL TRAVELS INDIA LIMITED,NON A/C Seater Semi Sleeper (2+1),21:30,06h 00m,03:30,3.2,INR 400,11 Seats available
7,https://www.redbus.in/bus-tickets/guwahati-to-...,Guwahati to Dhubri,BLUE HILL TRAVELS INDIA LIMITED,NON A/C Seater Semi Sleeper (2+1),21:30,06h 00m,03:30,3.2,INR 400,11 Seats available
8,https://www.redbus.in/bus-tickets/north-lakhim...,North Lakhimpur to Sibsagar,BLUE HILL TRAVELS INDIA LIMITED,NON A/C Seater Semi Sleeper (2+1),21:30,06h 00m,03:30,3.2,INR 400,11 Seats available
9,https://www.redbus.in/bus-tickets/north-lakhim...,North Lakhimpur to Jorhat,BLUE HILL TRAVELS INDIA LIMITED,NON A/C Seater Semi Sleeper (2+1),21:30,06h 00m,03:30,3.2,INR 400,11 Seats available


In [98]:
ap_ts_ks_sb_wb_bs_hr_rs_as=pd.concat([ap_ts_ks_sb_wb_bs_hr_rs,result8], axis=0, ignore_index=True)
ap_ts_ks_sb_wb_bs_hr_rs_as

,route_link,route_title,bus_name,bus_type,departing_time,duration,reaching_time,rating,price,seats_available
0,https://www.redbus.in/bus-tickets/vijayawada-t...,Vijayawada to Hyderabad,APSRTC - 35133,AMARAVATHI (VOLVO / SCANIA A.C Multi Axle),17:00,05h 50m,22:50,3.3,INR 720,33 Seats available
1,https://www.redbus.in/bus-tickets/hyderabad-to...,Hyderabad to Vijayawada,APSRTC - 35085,DOLPHIN CRUISE (VOLVO / SCANIA A.C Multi Axle),18:30,06h 00m,00:30,3.9,INR 720,38 Seats available
2,https://www.redbus.in/bus-tickets/kakinada-to-...,Kakinada to Visakhapatnam,APSRTC - 3613,Super Luxury (Non AC Seater 2+2 Push Back),19:00,07h 42m,02:42,3.1,INR 469,33 Seats available
3,https://www.redbus.in/bus-tickets/visakhapatna...,Visakhapatnam to Kakinada,APSRTC - 9316,Super Luxury (Non AC Seater 2+2 Push Back),20:20,07h 50m,04:10,4.1,INR 469,16 Seats available
4,https://www.redbus.in/bus-tickets/chittoor-and...,Chittoor (Andhra Pradesh) to Bangalore,APSRTC - 35189,DOLPHIN CRUISE (VOLVO / SCANIA A.C Multi Axle),20:30,04h 35m,01:05,3.8,INR 670,27 Seats available
...,...,...,...,...,...,...,...,...,...,...
846,https://www.redbus.in/bus-tickets/north-lakhim...,North Lakhimpur to Jorhat,Himalayan Travels,NON A/C Seater (2+2),20:30,06h 00m,02:30,N/A,450,24 Seats available
847,https://www.redbus.in/bus-tickets/north-lakhim...,North Lakhimpur to Jorhat,KIRAN TRAVELS,A/C Seater (2+1),20:40,03h 50m,00:30,3.1,540,13 Seats available
848,https://www.redbus.in/bus-tickets/north-lakhim...,North Lakhimpur to Jorhat,Pranjit Travels (Under ASTC),Bharat Benz A/C Seater (2+1),21:30,05h 00m,02:30,4.1,INR 600,9 Seats available
849,https://www.redbus.in/bus-tickets/north-lakhim...,North Lakhimpur to Jorhat,Megha Transport,Bharat Benz NON A/C Seater / Sleeper (2+1),19:30,02h 45m,22:15,3.6,INR 500,22 Seats available


In [126]:
final=pd.DataFrame(ap_ts_ks_sb_wb_bs_hr_rs_as)
final

,route_link,route_title,bus_name,bus_type,departing_time,duration,reaching_time,rating,price,seats_available
0,https://www.redbus.in/bus-tickets/vijayawada-t...,Vijayawada to Hyderabad,APSRTC - 35133,AMARAVATHI (VOLVO / SCANIA A.C Multi Axle),17:00,05h 50m,22:50,3.3,INR 720,33 Seats available
1,https://www.redbus.in/bus-tickets/hyderabad-to...,Hyderabad to Vijayawada,APSRTC - 35085,DOLPHIN CRUISE (VOLVO / SCANIA A.C Multi Axle),18:30,06h 00m,00:30,3.9,INR 720,38 Seats available
2,https://www.redbus.in/bus-tickets/kakinada-to-...,Kakinada to Visakhapatnam,APSRTC - 3613,Super Luxury (Non AC Seater 2+2 Push Back),19:00,07h 42m,02:42,3.1,INR 469,33 Seats available
3,https://www.redbus.in/bus-tickets/visakhapatna...,Visakhapatnam to Kakinada,APSRTC - 9316,Super Luxury (Non AC Seater 2+2 Push Back),20:20,07h 50m,04:10,4.1,INR 469,16 Seats available
4,https://www.redbus.in/bus-tickets/chittoor-and...,Chittoor (Andhra Pradesh) to Bangalore,APSRTC - 35189,DOLPHIN CRUISE (VOLVO / SCANIA A.C Multi Axle),20:30,04h 35m,01:05,3.8,INR 670,27 Seats available
...,...,...,...,...,...,...,...,...,...,...
846,https://www.redbus.in/bus-tickets/north-lakhim...,North Lakhimpur to Jorhat,Himalayan Travels,NON A/C Seater (2+2),20:30,06h 00m,02:30,N/A,450,24 Seats available
847,https://www.redbus.in/bus-tickets/north-lakhim...,North Lakhimpur to Jorhat,KIRAN TRAVELS,A/C Seater (2+1),20:40,03h 50m,00:30,3.1,540,13 Seats available
848,https://www.redbus.in/bus-tickets/north-lakhim...,North Lakhimpur to Jorhat,Pranjit Travels (Under ASTC),Bharat Benz A/C Seater (2+1),21:30,05h 00m,02:30,4.1,INR 600,9 Seats available
849,https://www.redbus.in/bus-tickets/north-lakhim...,North Lakhimpur to Jorhat,Megha Transport,Bharat Benz NON A/C Seater / Sleeper (2+1),19:30,02h 45m,22:15,3.6,INR 500,22 Seats available


In [127]:
final=final.ffill()
final

,route_link,route_title,bus_name,bus_type,departing_time,duration,reaching_time,rating,price,seats_available
0,https://www.redbus.in/bus-tickets/vijayawada-t...,Vijayawada to Hyderabad,APSRTC - 35133,AMARAVATHI (VOLVO / SCANIA A.C Multi Axle),17:00,05h 50m,22:50,3.3,INR 720,33 Seats available
1,https://www.redbus.in/bus-tickets/hyderabad-to...,Hyderabad to Vijayawada,APSRTC - 35085,DOLPHIN CRUISE (VOLVO / SCANIA A.C Multi Axle),18:30,06h 00m,00:30,3.9,INR 720,38 Seats available
2,https://www.redbus.in/bus-tickets/kakinada-to-...,Kakinada to Visakhapatnam,APSRTC - 3613,Super Luxury (Non AC Seater 2+2 Push Back),19:00,07h 42m,02:42,3.1,INR 469,33 Seats available
3,https://www.redbus.in/bus-tickets/visakhapatna...,Visakhapatnam to Kakinada,APSRTC - 9316,Super Luxury (Non AC Seater 2+2 Push Back),20:20,07h 50m,04:10,4.1,INR 469,16 Seats available
4,https://www.redbus.in/bus-tickets/chittoor-and...,Chittoor (Andhra Pradesh) to Bangalore,APSRTC - 35189,DOLPHIN CRUISE (VOLVO / SCANIA A.C Multi Axle),20:30,04h 35m,01:05,3.8,INR 670,27 Seats available
...,...,...,...,...,...,...,...,...,...,...
846,https://www.redbus.in/bus-tickets/north-lakhim...,North Lakhimpur to Jorhat,Himalayan Travels,NON A/C Seater (2+2),20:30,06h 00m,02:30,N/A,450,24 Seats available
847,https://www.redbus.in/bus-tickets/north-lakhim...,North Lakhimpur to Jorhat,KIRAN TRAVELS,A/C Seater (2+1),20:40,03h 50m,00:30,3.1,540,13 Seats available
848,https://www.redbus.in/bus-tickets/north-lakhim...,North Lakhimpur to Jorhat,Pranjit Travels (Under ASTC),Bharat Benz A/C Seater (2+1),21:30,05h 00m,02:30,4.1,INR 600,9 Seats available
849,https://www.redbus.in/bus-tickets/north-lakhim...,North Lakhimpur to Jorhat,Megha Transport,Bharat Benz NON A/C Seater / Sleeper (2+1),19:30,02h 45m,22:15,3.6,INR 500,22 Seats available


In [129]:
final['rating']=final['rating'].dropna()
final['rating']

0      3.3
1      3.9
2      3.1
3      4.1
4      3.8
      ... 
846    N/A
847    3.1
848    4.1
849    3.6
850    2.8
Name: rating, Length: 851, dtype: object

In [110]:
final['rating'].unique()

array(['3.3', '3.9', '3.1', '4.1', '3.8', '4.0', '4.4', '3.2', '3.0',
       '4.7', '4.6', '4.5', '2.8', '3.4', '3.5', '4.3', '1.8', '3.7',
       '2.9', '3.6', '2.4', '2.5', '1.9', '2.6', '2.3', '2.7', '4.2',
       '1.5', '2.1', 0.0, '1.2', '4.8', '2.0', '1.0', '1.7', '4.9', '2.2',
       '5.0', '1.6', '1.4', '1.3', '1.1', 'N/A'], dtype=object)

In [106]:
final['route_title'].unique()

array(['Vijayawada to Hyderabad', 'Hyderabad to Vijayawada',
       'Kakinada to Visakhapatnam', 'Visakhapatnam to Kakinada',
       'Chittoor (Andhra Pradesh) to Bangalore', 'Kadapa to Bangalore',
       'Anantapur (andhra pradesh) to Bangalore', 'Tirupati to Bangalore',
       'Visakhapatnam to Vijayawada', 'Ongole to Hyderabad',
       'Burdwan to Kolkata', 'Kolkata to Burdwan',
       'Durgapur (West Bengal) to Kolkata', 'Kolkata to Haldia',
       'Haldia to Kolkata', 'Kolkata to Durgapur (West Bengal)',
       'Kolkata to Arambagh (West Bengal)', 'Digha to Kolkata',
       'Kolkata to Digha', 'Kolkata to Bankura', 'Mandarmani to Kolkata',
       'Kolkata to Mandarmani', 'Kolkata to Bakkhali',
       'Patna (Bihar) to Bettiah', 'Gopalganj (Bihar) to Delhi',
       'Patna (Bihar) to Motihari', 'Delhi to Motihari',
       'Bettiah to Patna (Bihar)', 'Motihari to Delhi',
       'Patna (Bihar) to Balmiki Nagar (bihar)',
       'Balmiki Nagar (bihar) to Patna (Bihar)',
       'Patna (B

In [107]:
df_list=final.values.tolist()
df_list

[['https://www.redbus.in/bus-tickets/vijayawada-to-hyderabad',
  'Vijayawada to Hyderabad',
  'APSRTC - 35133',
  'AMARAVATHI (VOLVO / SCANIA A.C Multi Axle)',
  '17:00',
  '05h 50m',
  '22:50',
  '3.3',
  'INR 720',
  '33 Seats available'],
 ['https://www.redbus.in/bus-tickets/hyderabad-to-vijayawada',
  'Hyderabad to Vijayawada',
  'APSRTC - 35085',
  'DOLPHIN CRUISE (VOLVO / SCANIA A.C Multi Axle)',
  '18:30',
  '06h 00m',
  '00:30',
  '3.9',
  'INR 720',
  '38 Seats available'],
 ['https://www.redbus.in/bus-tickets/kakinada-to-visakhapatnam',
  'Kakinada to Visakhapatnam',
  'APSRTC - 3613',
  'Super Luxury (Non AC Seater 2+2 Push Back)',
  '19:00',
  '07h 42m',
  '02:42',
  '3.1',
  'INR 469',
  '33 Seats available'],
 ['https://www.redbus.in/bus-tickets/visakhapatnam-to-kakinada',
  'Visakhapatnam to Kakinada',
  'APSRTC - 9316',
  'Super Luxury (Non AC Seater 2+2 Push Back)',
  '20:20',
  '07h 50m',
  '04:10',
  '4.1',
  'INR 469',
  '16 Seats available'],
 ['https://www.redbus.

In [130]:
final['bus_type'].unique()

array(['AMARAVATHI (VOLVO / SCANIA A.C Multi Axle)',
       'DOLPHIN CRUISE (VOLVO / SCANIA A.C Multi Axle)',
       'Super Luxury (Non AC Seater 2+2 Push Back)',
       'STAR LINER(NON-AC SLEEPER 2+1)', 'VENNELA (A.C. SLEEPER)',
       'INDRA(A.C. Seater)', 'A/C Seater / Sleeper (2+1)',
       'Electric A/C Seater (2+2)', 'Scania AC Multi Axle Sleeper (2+1)',
       'METRO LUXURY A/C', 'ULTRA DELUXE (NON-AC, 2+2 PUSH BACK)',
       'NON A/C Push Back (2+2)', 'A/C Sleeper (2+1)',
       'Non A/C Seater / Sleeper (2+1)', 'Express(Non AC Seater)',
       'NON A/C Seater (2+3)', 'NON A/C Sleeper (2+1)',
       'NON A/C Seater/ Sleeper (2+1)',
       'Volvo Multi-Axle I-Shift A/C Semi Sleeper (2+2)',
       'Volvo A/C B11R Multi Axle Semi Sleeper (2+2)',
       'Volvo Multi Axle A/C Sleeper I-Shift B11R (2+1)',
       'SAPTAGIRI EXPRESS', '', 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)',
       'RAJDHANI (A.C. Semi Sleeper)',
       'RAJADHANI AC (CONVERTED METRO LUXURY)', 'NON A/C Seater (2+2

In [131]:
final['price'].unique()

array(['INR 720', 'INR 469', 'INR 670', 'INR 607', 'INR 781', 'INR 436',
       'INR 567', 'INR 543', '450', 'INR 419', 'INR 528', 'INR 839',
       'INR 621', 'INR 449', '500', 'INR 472', 'INR 340', 'INR 281',
       'INR 383', 'INR 269', 'INR 250', 'INR 239', 'INR 399', 'INR 660',
       'INR 499', 'INR 545', 'INR 890', 'INR 990', 'INR 170', 'INR 186',
       'INR 215', 'INR 304', '427', '314', '332', 'INR 649', 'INR 439',
       'INR 412', 'INR 526', 'INR 467', 'INR 460', 'INR 300', 'INR 750',
       'INR 450', 'INR 650', 'INR 307', 'INR 700', 'INR 900', 'INR 800',
       'INR 1100', 'INR 314', 'INR 294', '', 'INR 694', 'INR 605',
       'INR 624', 'INR 602', 'INR 590', 'INR 555', 'INR 320', 'INR 389',
       'INR 600', 'INR 380', 'INR 400', 'INR 610', 'INR 442', 'INR 343',
       'INR 283', 'INR 500', 'INR 1039', 'INR 790', 'INR 376', 'INR 407',
       'INR 456', '764', 'INR 329', 'INR 350', 'INR 429', 'INR 520',
       '924', 'INR 504', 'INR 532', 'INR 533', 'INR 611', 'INR 550',


**MYSQL CONNECTION**

In [109]:
import pandas as pd
import mysql.connector

# Sample DataFrame (assuming 'final' is already defined and contains data)
# final = pd.DataFrame({...})

# Define default values for columns
default_values = {
    'route_link': 'Unknown',
    'route_title': 'Unknown',
    'bus_name': 'Unknown',
    'bus_type': 'Unknown',
    'departing_time': '00:00',
    'duration': '00h 00m',
    'reaching_time': '00:00',
    'rating': 0.0,
    'price': 'INR 0',
    'seats_available': '0 Seats available'
}

# Replace NaN or empty values with default values
for column in final.columns:
    final[column] = final[column].fillna(default_values.get(column))
    if final[column].dtype == 'object':
        final[column] = final[column].replace('', default_values.get(column))

# Convert DataFrame to list of tuples
df_list = final.values.tolist()

# Connect to MySQL
conn = mysql.connector.connect(
    host="127.0.0.1",
    user="root",
    password="Geethu@2003",
    database="redbus",
    port=3306
)
cursor = conn.cursor()

# Define insert query
query = """
INSERT INTO bus_routes (route_link, route_title, bus_name, bus_type, departing_time, duration, reaching_time, rating, price, seats_available)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
"""

# Insert data
try:
    for row in df_list:
        cursor.execute(query, row)
    conn.commit()
    print("Data inserted successfully")
except mysql.connector.Error as err:
    print(f"Error: {err}")
finally:
    cursor.close()
    conn.close()


Error: 1366 (HY000): Incorrect decimal value: 'N/A' for column 'rating' at row 1
